In [1]:
# ENHANCED MOTOR LEARNING ANALYSIS WITH STRIDE CHANGE DISTRIBUTION
# Incorporates the stride change distribution plotting functionality
# ==============================================================================

import os
import re
import pickle
import tempfile
import webbrowser
from collections import defaultdict
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Union

import numpy as np
import pandas as pd
from scipy.stats import pearsonr, mannwhitneyu, gaussian_kde
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (mean_squared_error, r2_score, accuracy_score, 
                             precision_score, recall_score, f1_score, 
                             roc_auc_score, confusion_matrix, roc_curve)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import seaborn as sns

from IPython.display import display, HTML
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from scipy.stats import gaussian_kde, pearsonr, mannwhitneyu, ttest_ind, f_oneway
import warnings
from tqdm import tqdm
import json
from datetime import datetime

In [2]:
# 1. CONFIGURATION
# ==============================================================================


class Config:
    """Enhanced configuration settings for the motor learning analysis."""
    
    def __init__(self, base_output_dir: str = 'motor_learning_output'):
        # Base directory for all outputs
        self.BASE_OUTPUT_DIR = Path(base_output_dir)
        self.BASE_OUTPUT_DIR.mkdir(exist_ok=True)
        
        # Create organized subdirectory structure
        self.FIGURES_DIR = self.BASE_OUTPUT_DIR / 'figures'
        self.INDIVIDUAL_PLOTS_DIR = self.FIGURES_DIR / 'individual_plots'
        self.POPULATION_PLOTS_DIR = self.FIGURES_DIR / 'population_plots'
        self.STATISTICAL_PLOTS_DIR = self.FIGURES_DIR / 'statistical_plots'
        self.REPORTS_DIR = self.BASE_OUTPUT_DIR / 'reports'
        self.EXPORTS_DIR = self.BASE_OUTPUT_DIR / 'exports'
        self.PROCESSED_DATA_DIR = self.BASE_OUTPUT_DIR / 'processed_data'
        
        # Create all directories
        for directory in [self.FIGURES_DIR, self.INDIVIDUAL_PLOTS_DIR, 
                         self.POPULATION_PLOTS_DIR, self.STATISTICAL_PLOTS_DIR,
                         self.REPORTS_DIR, self.EXPORTS_DIR, self.PROCESSED_DATA_DIR]:
            directory.mkdir(parents=True, exist_ok=True)
        
        # File processing parameters
        self.MIN_COMPLETE_STRIDES = 20
        self.PROCESSED_DATA_FILE = self.PROCESSED_DATA_DIR / 'processed_data.pkl'
        
        # Trial type mappings
        self.TRIAL_TYPE_MAPPING = {
            'primer': 'vis1',
            'trial': 'invis', 
            'vis': 'vis2',
            'pref': 'pref'
        }
        
        # Analysis parameters
        self.MOTOR_NOISE_STRIDES = 20
        self.MOTOR_NOISE_THRESHOLD = 0.3
        self.SUCCESS_RATE_THRESHOLD = 0.68
        self.TARGET_SIZE_THRESHOLD = 0.31
        self.MAX_STRIDES_THRESHOLD = 415
        
        # Visualization parameters
        self.FIGURE_DPI = 300
        self.ALPHA_LEVEL = 0.05
        self.AGE_BINS = [7, 10, 13, 16, 18]
        self.AGE_LABELS = ['7-10', '10-13', '13-16', '16-18']
        
        print(f"📁 Config initialized with base directory: {self.BASE_OUTPUT_DIR}")
        print(f"   📊 Figures: {self.FIGURES_DIR}")
        print(f"   📋 Reports: {self.REPORTS_DIR}")
        print(f"   💾 Exports: {self.EXPORTS_DIR}")

    def get_figure_path(self, filename: str, subdir: str = 'general') -> Path:
        """Get standardized figure path with automatic subdirectory creation."""
        if subdir == 'individual':
            target_dir = self.INDIVIDUAL_PLOTS_DIR
        elif subdir == 'population':
            target_dir = self.POPULATION_PLOTS_DIR
        elif subdir == 'statistical':
            target_dir = self.STATISTICAL_PLOTS_DIR
        else:
            target_dir = self.FIGURES_DIR
        
        target_dir.mkdir(parents=True, exist_ok=True)
        return target_dir / filename
    
    def get_report_path(self, filename: str) -> Path:
        """Get standardized report path."""
        return self.REPORTS_DIR / filename
    
    def get_export_path(self, filename: str) -> Path:
        """Get standardized export path."""
        return self.EXPORTS_DIR / filename

In [3]:
# 2. UTILITY FUNCTIONS
# ==============================================================================

class DataUtils:
    """Utility functions for data processing."""
    
    @staticmethod
    def load_and_validate_file(file_path: Path, required_cols: set = None) -> Optional[pd.DataFrame]:
        """Load and validate a single data file."""
        try:
            df = pd.read_csv(file_path, sep='\t')
            
            if required_cols and not required_cols.issubset(df.columns):
                return None
                
            # Basic cleaning
            if 'Stride Number' in df.columns:
                df['Stride Number'] = pd.to_numeric(df['Stride Number'], errors='coerce')
                df = df.dropna(subset=['Stride Number'])
                df = df.drop_duplicates(subset=['Stride Number'])
            
            return df if not df.empty else None
            
        except Exception as e:
            print(f"❌ Error loading {file_path.name}: {str(e)}")
            return None

    @staticmethod
    def detect_anomalies(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict]:
        """Detect and flag anomalies in stride data."""
        if df is None or df.empty:
            return df, {}

        df = df.copy()
        df['Anomalous'] = False
        anomalies = {}

        # Time-based anomalies
        time_col = next((col for col in ['Time', 'Timestamp', 'Time (s)'] 
                        if col in df.columns), None)
        if time_col:
            df[time_col] = pd.to_numeric(df[time_col], errors='coerce')
            time_diff = df[time_col].diff()
            jump_mask = time_diff > time_diff.quantile(0.99) * 5
            
            for idx in df.index[jump_mask.fillna(False)]:
                df.at[idx, 'Anomalous'] = True
                anomalies.setdefault(idx, []).append('time_jump')

        # Sum of gains and steps anomalies
        if 'Sum of gains and steps' in df.columns:
            high_mask = df['Sum of gains and steps'] > 4
            zero_mask = df['Sum of gains and steps'] == 0

            for idx in df.index[high_mask]:
                df.at[idx, 'Anomalous'] = True
                anomalies.setdefault(idx, []).append('sum_gain_step_high')
            
            for idx in df.index[zero_mask]:
                df.at[idx, 'Anomalous'] = True
                anomalies.setdefault(idx, []).append('sum_gain_step_zero')

        # Duplicate rows
        duplicated_mask = df.duplicated()
        for idx in df.index[duplicated_mask]:
            df.at[idx, 'Anomalous'] = True
            anomalies.setdefault(idx, []).append('duplicate_row')

        return df, anomalies

In [4]:
# 3. TRIAL PROCESSING
# ==============================================================================

class TrialProcessor:
    """Handles loading, combining, and processing of trial data."""
    
    def __init__(self, debug: bool = True):
        self.debug = debug
        
    def find_and_combine_trial_files(self, subject_dir: Path, trial_prefix: str) -> Optional[pd.DataFrame]:
        """Find and combine trial files for a given trial type."""
        all_files = sorted(subject_dir.glob(f"{trial_prefix}*.txt"))
        
        if not all_files:
            if self.debug:
                print(f"  ⚠️ No files found for {trial_prefix}")
            return None
        
        # Special handling for preference trials
        if trial_prefix == 'pref':
            return self._handle_pref_trial(all_files)
        
        # Single file case
        if len(all_files) == 1:
            return DataUtils.load_and_validate_file(all_files[0])
        
        # Multiple files - combine them
        return self._combine_trial_fragments(all_files)
    
    def _handle_pref_trial(self, files: List[Path]) -> Optional[pd.DataFrame]:
        """Handle preference trial - select largest file."""
        largest_file = max(files, key=lambda f: f.stat().st_size)
        if self.debug and len(files) > 1:
            print(f"  ⚡ pref trial - selected largest of {len(files)} files")
        return DataUtils.load_and_validate_file(largest_file)
    
    def _combine_trial_fragments(self, files: List[Path]) -> Optional[pd.DataFrame]:
        """Combine multiple trial fragments intelligently."""
        file_info = []
        
        for f in files:
            try:
                with open(f, 'r') as file:
                    header = file.readline().strip().split('\t')
                    stride_col = next((i for i, col in enumerate(header) 
                                     if 'stride' in col.lower() and 
                                     ('num' in col.lower() or 'no' in col.lower())), None)
                    
                    if stride_col is None:
                        continue
                    
                    # Get stride range
                    lines = file.readlines()
                    first_stride = float(lines[0].split('\t')[stride_col])
                    last_stride = float(lines[-1].split('\t')[stride_col])
                    
                    file_info.append({
                        'path': f,
                        'first': first_stride,
                        'last': last_stride,
                        'size': f.stat().st_size
                    })
            except Exception:
                continue
        
        if not file_info:
            return DataUtils.load_and_validate_file(max(files, key=lambda f: f.stat().st_size))
        
        # Find best continuous sequence
        file_info.sort(key=lambda x: x['first'])
        best_sequence = self._find_best_sequence(file_info)
        
        if len(best_sequence) >= 2:
            return self._merge_files([f['path'] for f in best_sequence])
        
        # Fallback to largest file
        return DataUtils.load_and_validate_file(max(file_info, key=lambda x: x['size'])['path'])
    
    def _find_best_sequence(self, file_info: List[Dict]) -> List[Dict]:
        """Find the best continuous sequence of files."""
        best_sequence = []
        current_sequence = [file_info[0]]
        
        for file_data in file_info[1:]:
            if file_data['first'] == current_sequence[-1]['last'] + 1:
                current_sequence.append(file_data)
            else:
                if len(current_sequence) > len(best_sequence):
                    best_sequence = current_sequence
                current_sequence = [file_data]
        
        return max([best_sequence, current_sequence], key=len)
    
    def _merge_files(self, file_paths: List[Path]) -> Optional[pd.DataFrame]:
        """Merge multiple files into a single DataFrame."""
        dfs = []
        for f in file_paths:
            df = DataUtils.load_and_validate_file(f)
            if df is not None:
                dfs.append(df)
        
        if not dfs:
            return None
        
        combined = pd.concat(dfs, ignore_index=True)
        
        # Clean and sort
        if 'Stride Number' in combined.columns:
            combined = combined.sort_values('Stride Number')
            combined = combined.drop_duplicates('Stride Number')
        
        return combined

In [5]:
# 4. MAIN DATA MANAGER
# ==============================================================================

class MotorLearningDataManager:
    """Updated to use centralized config."""
    
    def __init__(self, metadata_path: str, data_root_dir: str, 
                 config: Config = None, force_reprocess: bool = False, debug: bool = True):
        self.metadata_path = metadata_path
        self.data_root_dir = data_root_dir
        self.debug = debug
        
        # KEY CHANGE: Use provided config or create default
        self.config = config if config else Config()
        
        # Initialize components (unchanged)
        self.trial_processor = TrialProcessor(debug=debug)
        
        # Data storage (unchanged)
        self.metadata = None
        self.processed_data = {}
        
        # KEY CHANGE: Use config for processed data file path
        if not force_reprocess and self.config.PROCESSED_DATA_FILE.exists():
            self._load_processed_data()
        else:
            self._process_all_data()
            self._save_processed_data()
    
    def _load_processed_data(self):
        """Updated to use config path."""
        with open(self.config.PROCESSED_DATA_FILE, 'rb') as f:
            self.processed_data = pickle.load(f)
            
        # Rebuild metadata DataFrame (unchanged)
        self.metadata = pd.DataFrame.from_dict(
            {subj: data['metadata'] for subj, data in self.processed_data.items()}, 
            orient='index'
        )
    
    def _save_processed_data(self):
        """Updated to use config path."""
        with open(self.config.PROCESSED_DATA_FILE, 'wb') as f:
            pickle.dump(self.processed_data, f)
    
    def _process_all_data(self):
        """Process all subject data."""
        self._load_metadata()
        total_subjects = len(self.metadata)
        
        if self.debug:
            print(f"🔄 Processing {total_subjects} subjects...")
        
        for i, (_, row) in enumerate(self.metadata.iterrows(), 1):
            subject_id = row['ID']
            if self.debug:
                print(f"\n[{i}/{total_subjects}] Processing {subject_id}...")
            
            subject_data = self._process_subject_data(subject_id, row)
            if subject_data:
                self.processed_data[subject_id] = subject_data
    
    def _load_metadata(self):
        """Load and clean metadata."""
        self.metadata = pd.read_csv(self.metadata_path)
        self.metadata['DOB'] = pd.to_datetime(self.metadata['DOB'], errors='coerce')
        self.metadata['Session Date'] = pd.to_datetime(self.metadata['Session Date'], errors='coerce')
        self.metadata = self.metadata.dropna(subset=['ID', 'age_months'])
    
    def _process_subject_data(self, subject_id: str, metadata_row: pd.Series) -> Optional[Dict]:
        """Process data for a single subject."""
        subject_dir = Path(self.data_root_dir) / subject_id
        if not subject_dir.exists():
            if self.debug:
                print(f"❌ Directory not found: {subject_dir}")
            return None
        
        trial_data = {}
        
        for original_type in ['primer', 'trial', 'vis', 'pref']:
            try:
                # Load trial data
                df = self.trial_processor.find_and_combine_trial_files(subject_dir, original_type)
                
                if df is not None:
                    # Process the data
                    processed_df, anomalies = self._process_trial_data(df, original_type)
                    
                    # Store with mapped name
                    new_type = self.config.TRIAL_TYPE_MAPPING[original_type]
                    trial_data[new_type] = {
                        'data': processed_df,
                        'anomalies': anomalies
                    }
                    
            except Exception as e:
                print(f"Error processing {subject_id}/{original_type}: {str(e)}")
                continue
        
        return {
            'metadata': metadata_row.to_dict(),
            'trial_data': trial_data
        } if trial_data else None
    
    def _process_trial_data(self, df: pd.DataFrame, trial_type: str) -> Tuple[pd.DataFrame, Dict]:
        """Process trial data and calculate metrics."""
        if df is None or df.empty:
            return None, {}
        
        # Skip processing for pref trials (just clean duplicates)
        if trial_type == 'pref':
            df = df.drop_duplicates(subset='Left heel strike', keep='last')
            return df, {}
        
        # Validate required columns
        required_cols = ['Stride Number', 'Success', 'Upper bound success', 
                        'Lower bound success', 'Constant']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            print(f"Missing required columns: {missing_cols}")
            return None, {}
        
        try:
            # Process trial data
            df = df.sort_values('Stride Number')
            df['Target size'] = df['Upper bound success'] - df['Lower bound success']
            df = df.drop_duplicates(subset='Stride Number', keep='last')
            
            # Scale sum of gains and steps
            if 'Sum of gains and steps' in df.columns:
                df['Sum of gains and steps'] = 1.5 * df['Sum of gains and steps']
            
            # Detect anomalies
            df, anomalies = DataUtils.detect_anomalies(df)
            
            return df, anomalies
            
        except Exception as e:
            print(f"Error processing {trial_type} data: {str(e)}")
            return None, {}
    
    def get_trial_df(self, trial_dict: Dict) -> Optional[pd.DataFrame]:
        """Get trial data from trial dictionary."""
        if trial_dict and 'data' in trial_dict:
            return trial_dict['data']
        return None
    
    def filter_trials(self, max_target_size=None, min_age=None, max_age=None, 
                     required_trial_types=None, min_strides=None, max_strides=None):
        """Filter trials based on specified criteria."""
        filtered_data = {}
        
        for subject_id, subject_data in self.processed_data.items():
            # SIMPLIFIED AGE FILTERING - always use age_months/12
            age = subject_data['metadata']['age_months'] / 12
            if min_age is not None and age < min_age:
                continue
            if max_age is not None and age > max_age:
                continue
            
            # Check required trial types
            if required_trial_types:
                missing_trials = [
                    t for t in required_trial_types 
                    if t not in subject_data['trial_data'] or 
                       subject_data['trial_data'][t] is None or
                       subject_data['trial_data'][t]['data'] is None
                ]
                if missing_trials:
                    continue
            
            # Check other criteria
            valid_subject = True
            filtered_trial_data = {}
            
            for trial_type, trial_dict in subject_data['trial_data'].items():
                if trial_dict and trial_dict['data'] is not None:
                    df = trial_dict['data']
                    
                    # Apply filters
                    if (max_target_size is not None and 
                        'Target size' in df.columns and 
                        df['Target size'].min() > max_target_size):
                        valid_subject = False
                        break
                    
                    n_strides = len(df)
                    if ((min_strides is not None and n_strides < min_strides) or
                        (max_strides is not None and n_strides > max_strides)):
                        valid_subject = False
                        break
                    
                    # Include valid trial
                    filtered_trial_data[trial_type] = {
                        'data': df.copy(),
                        'anomalies': trial_dict['anomalies'].copy()
                    }
            
            if valid_subject and filtered_trial_data:
                filtered_data[subject_id] = {
                    'metadata': subject_data['metadata'].copy(),
                    'trial_data': filtered_trial_data
                }
        
        # Create new instance with filtered data
        new_instance = MotorLearningDataManager.__new__(MotorLearningDataManager)
        new_instance.config = self.config
        new_instance.trial_processor = self.trial_processor
        new_instance.metadata_path = self.metadata_path
        new_instance.data_root_dir = self.data_root_dir
        new_instance.debug = self.debug
        new_instance.processed_data = filtered_data
        new_instance.metadata = pd.DataFrame.from_dict(
            {subj: data['metadata'] for subj, data in filtered_data.items()}, 
            orient='index'
        )
        
        return new_instance
    
    def get_trial_data(self, subject_id: str, trial_type: str) -> Optional[pd.DataFrame]:
        """Get trial data for a specific subject and trial type."""
        try:
            trial_dict = self.processed_data[subject_id]['trial_data'][trial_type]
            return trial_dict['data'] if trial_dict else None
        except KeyError:
            return None
    
    def print_summary(self):
        """Print summary statistics of the dataset."""
        print(f"📊 Dataset Summary:")
        print(f"Total subjects: {len(self.processed_data)}")
        
        if self.metadata is not None:
            # SIMPLIFIED AGE DISPLAY - always use age_months/12
            ages = self.metadata['age_months'] / 12
            print(f"Age range: {ages.min():.1f} - {ages.max():.1f} years")
            print(f"Mean age: {ages.mean():.1f} years")
        
        # Trial type counts
        trial_counts = defaultdict(int)
        for subject_data in self.processed_data.values():
            for trial_type in subject_data['trial_data'].keys():
                trial_counts[trial_type] += 1
        
        print(f"Trial type counts: {dict(trial_counts)}")

In [6]:
# 5. METRICS CALCULATOR
# ==============================================================================

class MetricsCalculator:
    """Updated to use config thresholds."""
    
    def __init__(self, data_manager: MotorLearningDataManager, motor_noise_strides: int = 10):
        self.data_manager = data_manager
        # KEY CHANGE: Get config from data_manager
        self.config = data_manager.config
        self.motor_noise_strides = self.config.MOTOR_NOISE_STRIDES
    
    def calculate_all_metrics(self) -> pd.DataFrame:
        """Calculate comprehensive performance metrics for all subjects."""
        results = []
        
        print(f"🧮 Calculating metrics for {len(self.data_manager.processed_data)} subjects...")
        
        for i, (subject_id, data) in enumerate(self.data_manager.processed_data.items(), 1):
            if i % 20 == 0:
                print(f"   Processed {i}/{len(self.data_manager.processed_data)} subjects...")
            
            try:
                subject_result = self._calculate_subject_metrics(subject_id, data)
                if subject_result:
                    results.append(subject_result)
            except Exception as e:
                print(f"   ⚠️ Error processing {subject_id}: {str(e)}")
                continue
        
        if not results:
            print("❌ No valid metrics calculated!")
            return pd.DataFrame()
        
        df = pd.DataFrame(results).infer_objects()
        print(f"✅ Successfully calculated metrics for {len(df)} subjects")
        
        # Print age and column summary
        if 'age' in df.columns:
            ages = df['age'].dropna()
            print(f"📊 Age range: {ages.min():.1f} - {ages.max():.1f} years")
        
        sr_cols = [col for col in df.columns if '_sr_' in col]
        print(f"🎯 Success rate columns created: {sr_cols}")
        
        return df
    
    def _calculate_subject_metrics(self, subject_id: str, subject_data: Dict) -> Optional[Dict]:
        """Calculate metrics for a single subject with simplified age handling."""
        
        # SIMPLIFIED: Only store age as age_months/12, call it 'age'
        result = {
            'ID': subject_id,
            'age': subject_data['metadata'].get('age_months', np.nan) / 12,  # Single age field
            'session_date': subject_data['metadata'].get('Session Date')
        }
        
        # Process each trial type
        for trial_type in ['vis1', 'invis', 'vis2']:
            try:
                trial_dict = subject_data['trial_data'].get(trial_type)
                df = trial_dict['data'] if trial_dict else None
                
                if df is None or df.empty or 'Success' not in df.columns:
                    continue
                
                # Calculate metrics for both conditions
                for condition in ['max', 'min']:
                    period_data, indices = self._get_period_data(df, condition)
                    if period_data is not None and not period_data.empty:
                        metrics = self._calculate_period_metrics(period_data, trial_type, condition)
                        result.update(metrics)
                        result[f'{trial_type}_{condition}_const_indices'] = indices
                
                # Add trial metadata - ONLY if df is not None
                if df is not None:
                    result.update({
                        f'{trial_type}_min_target_size': df['Target size'].min() if 'Target size' in df.columns else None,
                        f'{trial_type}_max_constant': df['Constant'].max() if 'Constant' in df.columns else None,
                        f'{trial_type}_min_constant': df['Constant'].min() if 'Constant' in df.columns else None
                    })
                    
                    # Order information for invis trials
                    if trial_type == 'invis':
                        result.update(self._calculate_condition_order(df))
                        
            except Exception as e:
                continue
        
        # Process preference trial
        try:
            pref_metrics = self._calculate_preference_metrics(subject_data['trial_data'].get('pref'))
            result.update(pref_metrics)
        except Exception as e:
            result.update({'mot_noise': None, 'pref_asymmetry': None})
        
        return result
    
    def _get_period_data(self, df: pd.DataFrame, condition: str, length: int = 20):
        """Extract data for specific condition period."""
        try:
            if df is None or df.empty:
                return None, None
            
            if 'Target size' not in df.columns or 'Constant' not in df.columns:
                return None, None
            
            min_target = df['Target size'].min()
            target_tolerance = 0.001
            
            min_target_periods = df[df['Target size'] <= min_target + target_tolerance]
            if min_target_periods.empty:
                return None, None
            
            const_value = (min_target_periods['Constant'].max() if condition == 'max'
                          else min_target_periods['Constant'].min())
            
            period_data = min_target_periods[
                np.isclose(min_target_periods['Constant'], const_value, rtol=1e-5)
            ]
            
            if period_data.empty:
                return None, None
            
            return period_data.tail(length), period_data.index
            
        except Exception as e:
            return None, None
    
    def _calculate_period_metrics(self, period_data: pd.DataFrame, trial_type: str, condition: str) -> Dict:
        """Calculate metrics for a specific period with naming."""
        metrics = {}
        
        try:
            # CORRECTED: Use the format that analysis expects
            # Success rate - THE KEY METRIC  
            metrics[f'{trial_type}_sr_{condition}_const'] = period_data['Success'].mean()
            
            # Other metrics
            if 'Sum of gains and steps' in period_data.columns:
                sogs = period_data['Sum of gains and steps']
                metrics[f'{trial_type}_sd_{condition}_const'] = sogs.std()
                metrics[f'{trial_type}_msl_{condition}_const'] = sogs.mean()
                metrics[f'{trial_type}_error_{condition}_const'] = (sogs - period_data['Constant']).mean()
            
            # Asymmetry
            if all(col in period_data.columns for col in ['Right step length', 'Left step length']):
                right_steps = period_data['Right step length']
                left_steps = period_data['Left step length']
                denominator = right_steps + left_steps
                
                valid_mask = denominator != 0
                if valid_mask.any():
                    asymmetry = ((right_steps - left_steps) / denominator).abs()[valid_mask].mean()
                    metrics[f'{trial_type}_asymmetry_{condition}_const'] = asymmetry
            
            # Strides between successes
            strides_between = self._calculate_strides_between_successes(period_data)
            if strides_between is not None:
                metrics[f'{trial_type}_strides_between_success_{condition}_const'] = strides_between
            
        except Exception as e:
            pass
        
        return metrics
    
    def _calculate_strides_between_successes(self, df: pd.DataFrame) -> Optional[float]:
        """Calculate average strides between successful trials."""
        try:
            if df is None or 'Success' not in df.columns:
                return None
            
            df = df.reset_index(drop=True)
            success_positions = df.index[df['Success'] == 1].tolist()
            
            if len(success_positions) < 2:
                return None
            
            return np.mean(np.diff(success_positions))
            
        except Exception as e:
            return None
    
    def _calculate_condition_order(self, df: pd.DataFrame) -> Dict:
        """Determine which condition came first for invis trials."""
        try:
            all_max_indices = df.index[df['Constant'] == df['Constant'].max()].tolist()
            all_min_indices = df.index[df['Constant'] == df['Constant'].min()].tolist()
            
            if all_max_indices and all_min_indices:
                first_max = min(all_max_indices)
                first_min = min(all_min_indices)
                return {
                    'invis_max_first': first_max < first_min,
                    'invis_min_first': first_min < first_max
                }
            
            return {'invis_max_first': False, 'invis_min_first': False}
            
        except Exception as e:
            return {'invis_max_first': False, 'invis_min_first': False}
    
    def _calculate_preference_metrics(self, pref_trial_dict: Optional[Dict]) -> Dict:
        """Calculate metrics from preference trial with configurable stride count."""
        metrics = {'mot_noise': None, 'pref_asymmetry': None}
        
        try:
            if not pref_trial_dict or pref_trial_dict.get('data') is None:
                return metrics
            
            pref_df = pref_trial_dict['data']
            if pref_df is None or pref_df.empty:
                return metrics
            
            # Check for required columns
            if not all(col in pref_df.columns for col in ['Right step length', 'Left step length']):
                return metrics
            
            # Get non-zero steps
            right_steps = pref_df['Right step length']
            left_steps = pref_df['Left step length']
            
            # Remove zeros and NaNs
            right_clean = right_steps[(right_steps != 0) & (right_steps.notna())]
            left_clean = left_steps[(left_steps != 0) & (left_steps.notna())]
            
            # Check if we have enough data
            min_required = self.motor_noise_strides
            if len(right_clean) < min_required or len(left_clean) < min_required:
                print(f"   ⚠️ Insufficient data for motor noise: need {min_required}, have R:{len(right_clean)}, L:{len(left_clean)}")
                return metrics
            
            # Calculate motor noise using configurable number of strides
            final_right = right_clean.iloc[-1]
            final_left = left_clean.iloc[-1]
            
            if final_right <= 0 or final_left <= 0:
                return metrics
            
            # Normalize steps
            norm_right = right_clean / final_right
            norm_left = left_clean / final_left
            
            # Calculate sum of normalized steps
            min_length = min(len(norm_right), len(norm_left))
            if min_length < min_required:
                return metrics
            
            sum_steps = norm_right.iloc[:min_length] + norm_left.iloc[:min_length]
            
            # Motor noise from last N points (configurable)
            if len(sum_steps) >= self.motor_noise_strides:
                noise = sum_steps.tail(self.motor_noise_strides).std()
                if not pd.isna(noise) and noise > 0:
                    metrics['mot_noise'] = noise
            
            # Calculate step length asymmetry using same number of strides
            if len(right_clean) >= self.motor_noise_strides and len(left_clean) >= self.motor_noise_strides:
                last_n_right = right_clean.tail(self.motor_noise_strides) / final_right
                last_n_left = left_clean.tail(self.motor_noise_strides) / final_left
                
                if len(last_n_right) == len(last_n_left):
                    denominator = last_n_right.values + last_n_left.values
                    valid_mask = denominator != 0
                    
                    if valid_mask.any():
                        asymmetry_vals = np.abs((last_n_right.values - last_n_left.values) / denominator)[valid_mask]
                        if len(asymmetry_vals) > 0:
                            metrics['pref_asymmetry'] = np.mean(asymmetry_vals)
            
        except Exception as e:
            print(f"   ⚠️ Error calculating preference metrics: {e}")
            pass
        
        return metrics

In [7]:
# 6. STATISTICAL ANALYZER
# ==============================================================================

class StatisticalAnalyzer:
    """Updated to use config thresholds."""
    
    def __init__(self, metrics_df: pd.DataFrame, config: Config = None):
        self.metrics_df = metrics_df
        
        # KEY CHANGE: Accept config parameter
        self.config = config if config else Config()
        
        # Apply motor noise filter using config
        if 'mot_noise' in metrics_df.columns:
            self.filtered_df = metrics_df[metrics_df['mot_noise'] <= self.config.MOTOR_NOISE_THRESHOLD]
            print(f"📊 Filtered to {len(self.filtered_df)}/{len(metrics_df)} subjects (motor noise ≤ {self.config.MOTOR_NOISE_THRESHOLD})")
        else:
            self.filtered_df = metrics_df
    
    def run_regression_analysis(self, trial_type: str = 'invis', condition: str = 'max',
                               predictors: List[str] = None, model_type: str = 'linear') -> Dict:
        """Run regression analysis withtarget column naming."""
        
        if predictors is None:
            predictors = ['age', 'mot_noise', 'pref_asymmetry']
        
        # Filter available predictors
        available_predictors = [p for p in predictors if p in self.filtered_df.columns]
        
        # CORRECTED: Use the exact column format that exists
        target_col = f'{trial_type}_sr_{condition}_const'
        
        if target_col not in self.filtered_df.columns:
            available_cols = [col for col in self.filtered_df.columns if '_sr_' in col]
            raise ValueError(f"Target column {target_col} not found. Available: {available_cols}")
        
        # Prepare data
        valid_data = self.filtered_df[available_predictors + [target_col]].dropna()
        
        if len(valid_data) < 10:
            raise ValueError(f"Insufficient data: only {len(valid_data)} valid samples")
        
        X = valid_data[available_predictors]
        y = valid_data[target_col]
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Create model
        if model_type == 'linear':
            model = Pipeline([
                ('scaler', StandardScaler()),
                ('regressor', LinearRegression())
            ])
        elif model_type == 'random_forest':
            model = RandomForestRegressor(random_state=42)
        else:
            raise ValueError("model_type must be 'linear' or 'random_forest'")
        
        # Fit and predict
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        # Feature importance
        if model_type == 'random_forest':
            importances = dict(zip(available_predictors, model.feature_importances_))
        else:
            importances = dict(zip(available_predictors, 
                                 np.abs(model.named_steps['regressor'].coef_)))
        
        return {
            'model': model,
            'X_test': X_test,
            'y_test': y_test,
            'y_pred': y_pred,
            'metrics': {
                'r2': r2,
                'rmse': rmse,
                'n_samples': len(valid_data),
                'trial_type': trial_type,
                'condition': condition,
                'predictors': available_predictors
            },
            'feature_importances': importances
        }
    
    def run_classification_analysis(self, trial_type: str = 'invis', condition: str = 'max',
                                   threshold: float = 0.68, model_type: str = 'logistic') -> Dict:
        """Run binary classification withtarget column naming."""
        
        predictors = ['age', 'mot_noise']
        available_predictors = [p for p in predictors if p in self.filtered_df.columns]
        
        # CORRECTED: Use the exact column format that exists
        target_col = f'{trial_type}_sr_{condition}_const'
        
        if target_col not in self.filtered_df.columns:
            available_cols = [col for col in self.filtered_df.columns if '_sr_' in col]
            raise ValueError(f"Target column {target_col} not found. Available: {available_cols}")
        
        # Prepare data
        valid_data = self.filtered_df[available_predictors + [target_col]].dropna()
        
        if len(valid_data) < 10:
            raise ValueError(f"Insufficient data: only {len(valid_data)} valid samples")
        
        # Create binary target
        y = (valid_data[target_col] >= threshold).astype(int)
        X = valid_data[available_predictors]
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Create model
        if model_type == 'logistic':
            model = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', LogisticRegression(random_state=42))
            ])
        elif model_type == 'random_forest':
            model = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier(random_state=42))
            ])
        else:
            raise ValueError("model_type must be 'logistic' or 'random_forest'")
        
        # Fit and predict
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
        
        # Calculate metrics
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba),
            'confusion_matrix': confusion_matrix(y_test, y_pred),
            'n_samples': len(valid_data),
            'threshold': threshold,
            'trial_type': trial_type,
            'condition': condition,
            'predictors': available_predictors
        }
        
        # Feature importance
        if model_type == 'random_forest':
            importances = dict(zip(available_predictors, 
                                 model.named_steps['classifier'].feature_importances_))
        else:
            importances = dict(zip(available_predictors, 
                                 model.named_steps['classifier'].coef_[0]))
        
        return {
            'model': model,
            'X_test': X_test,
            'y_test': y_test,
            'y_pred': y_pred,
            'y_proba': y_proba,
            'metrics': metrics,
            'feature_importances': importances
        }
    
    def run_mixed_effects_analysis(self, trial_types: Union[str, List[str]] = 'all') -> object:
        """Run mixed-effects analysis withcolumn naming."""
        
        # Prepare long-format data
        long_rows = []
        
        if trial_types == 'all':
            trials_to_include = ['vis1', 'invis', 'vis2']
        elif isinstance(trial_types, str):
            trials_to_include = [trial_types]
        else:
            trials_to_include = trial_types
        
        for trial in trials_to_include:
            for condition in ['max_const', 'min_const']:
                # CORRECTED: Use the exact column format
                sr_col = f"{trial}_sr_{condition}"
                if sr_col in self.filtered_df.columns:
                    sub_df = self.filtered_df[['ID', sr_col, 'mot_noise', 'age']].copy()
                    sub_df = sub_df.rename(columns={sr_col: 'success_rate'})
                    sub_df['trial_type'] = trial
                    sub_df['condition'] = condition
                    long_rows.append(sub_df)
        
        if not long_rows:
            raise ValueError("No success rate data available")
        
        df_long = pd.concat(long_rows, ignore_index=True)
        df_long.dropna(subset=['success_rate', 'mot_noise', 'age'], inplace=True)
        
        # Fit model
        if len(trials_to_include) == 1:
            formula = 'success_rate ~ C(condition) + mot_noise + age'
        else:
            formula = 'success_rate ~ C(trial_type) + C(condition) + mot_noise + age'
        
        model = smf.ols(formula, data=df_long).fit()
        return model

In [8]:
# 8. ANALYSIS PIPELINE
# ==============================================================================
# ORIGINAL ANALYSIS CLASS (RESTORED)
# ==================================
# This version uses the fixed StandaloneEnhancedVisualizer

class MotorLearningAnalysis:
    """Updated to use centralized config and new visualizer."""
    
    def __init__(self, data_manager, metrics_df: pd.DataFrame):
        self.data_manager = data_manager
        self.metrics_df = metrics_df
        
        # KEY CHANGE: Get config from data_manager
        self.config = data_manager.config
        
        # KEY CHANGE: Pass config to analyzer
        self.analyzer = StatisticalAnalyzer(metrics_df, self.config)
        
        # KEY CHANGE: Use StandaloneEnhancedVisualizer with config
        self.visualizer = StandaloneEnhancedVisualizer(self)
    
    def run_comprehensive_analysis(self, save_all: bool = True) -> Dict:
        """Updated to use config for saving."""
        
        results = {}
        
        print("🔬 Running Comprehensive Motor Learning Analysis...")
        print(f"📁 Outputs will be saved to: {self.config.BASE_OUTPUT_DIR}")
        
        # Print age summary (unchanged)
        if 'age' in self.metrics_df.columns:
            ages = self.metrics_df['age'].dropna()
            print(f"📊 Age range: {ages.min():.1f} - {ages.max():.1f} years (n={len(ages)})")
        
        # 1. Regression Analysis (unchanged logic, but could use config thresholds)
        print("📊 Running regression analysis...")
        try:
            regression_results = self.analyzer.run_regression_analysis()
            results['regression'] = regression_results
            print(f"✅ Regression R² = {regression_results['metrics']['r2']:.3f}")
            
            for feature, importance in regression_results['feature_importances'].items():
                print(f"   {feature}: {importance:.4f}")
                
        except Exception as e:
            print(f"❌ Regression analysis failed: {e}")
        
        # 2. Classification Analysis (unchanged)
        print("🎯 Running classification analysis...")
        try:
            classification_results = self.analyzer.run_classification_analysis()
            results['classification'] = classification_results
            print(f"✅ Classification AUC = {classification_results['metrics']['roc_auc']:.3f}")
            print(f"   Accuracy = {classification_results['metrics']['accuracy']:.3f}")
        except Exception as e:
            print(f"❌ Classification analysis failed: {e}")
        
        # 3. Mixed Effects Analysis (unchanged)
        print("📈 Running mixed-effects analysis...")
        try:
            mixed_model = self.analyzer.run_mixed_effects_analysis()
            results['mixed_effects'] = mixed_model
            print(f"✅ Mixed-effects R² = {mixed_model.rsquared:.3f}")
        except Exception as e:
            print(f"❌ Mixed-effects analysis failed: {e}")
        
        # 4. Enhanced Visualizations - KEY CHANGE: Uses config-managed paths
        print("📊 Generating enhanced visualizations...")
        try:
            # The visualizer now automatically saves to config-managed directories
            self.visualizer.plot_population_analyses(save=save_all)
            print("✅ Enhanced visualizations complete")
            print(f"📁 Plots saved to: {self.config.POPULATION_PLOTS_DIR}")
        except Exception as e:
            print(f"❌ Visualization failed: {e}")
        
        print("🎉 Comprehensive analysis complete!")
        print(f"📁 All outputs in: {self.config.BASE_OUTPUT_DIR}")
        return results

In [9]:
# ENHANCED STANDALONE VISUALIZER WITH CONSOLIDATED PLOTTING METHODS
# ==============================================================================
# This consolidates all plotting methods from StreamlinedMotorLearningPipeline
# into the StandaloneEnhancedVisualizer class for better organization



class StandaloneEnhancedVisualizer:
    """
    Enhanced visualizer that consolidates ALL plotting functionality.
    
    Key improvements:
    - All plotting methods moved from StreamlinedMotorLearningPipeline
    - Enhanced population statistics with motor noise coloring
    - Age vs success rates by trial/condition
    - Age vs stride variability by trial/condition  
    - Mean stride length vs success rate by trial/condition
    - Motor noise vs success rates analysis
    - Age vs mean stride length and error analysis
    - Statistical analysis plots
    - Summary dashboard generation
    """
    
    def __init__(self, analysis_instance, config: 'Config' = None):
        """
        Initialize with your existing analysis instance.
        
        Parameters:
        -----------
        analysis_instance : Your existing MotorLearningAnalysis object
            Should have .data_manager, .metrics_df attributes
        config : Config object
            Configuration settings
        """
        self.analysis = analysis_instance
        self.metrics_df = analysis_instance.metrics_df
        self.data_manager = analysis_instance.data_manager
        
        # Use provided config or get from data_manager or create default
        if config:
            self.config = config
        elif hasattr(self.data_manager, 'config'):
            self.config = self.data_manager.config
        else:
            # Create minimal config if none available
            self.config = self._create_minimal_config()
        
        # Set up directory attributes properly
        self.individual_plots_dir = self.config.INDIVIDUAL_PLOTS_DIR
        self.population_plots_dir = self.config.POPULATION_PLOTS_DIR
        self.statistical_plots_dir = self.config.STATISTICAL_PLOTS_DIR
        
        # Apply motor noise filter
        if 'mot_noise' in self.metrics_df.columns:
            motor_noise_threshold = getattr(self.config, 'MOTOR_NOISE_THRESHOLD', 0.3)
            self.filtered_df = self.metrics_df[self.metrics_df['mot_noise'] <= motor_noise_threshold]
            print(f"📊 Using {len(self.filtered_df)}/{len(self.metrics_df)} subjects (motor noise ≤ {motor_noise_threshold})")
        else:
            self.filtered_df = self.metrics_df
            print(f"📊 Using all {len(self.filtered_df)} subjects")
        
        # Set colors
        self.colors = {
            'primary': '#667eea',
            'secondary': '#764ba2',
            'success': '#28a745',
            'warning': '#ffc107',
            'danger': '#dc3545',
            'vis1': '#1f77b4',
            'invis': '#ff7f0e', 
            'vis2': '#2ca02c'
        }
        
        print(f"✅ Enhanced visualizer initialized with consolidated plotting methods")
        print(f"📁 Individual plots: {self.individual_plots_dir}")
        print(f"📁 Population plots: {self.population_plots_dir}")
        print(f"📁 Statistical plots: {self.statistical_plots_dir}")

    def _create_minimal_config(self):
        """Create minimal config if none provided."""
        class MinimalConfig:
            def __init__(self):
                base_dir = Path('motor_learning_output')
                base_dir.mkdir(exist_ok=True)
                
                self.BASE_OUTPUT_DIR = base_dir
                self.INDIVIDUAL_PLOTS_DIR = base_dir / 'figures' / 'individual_plots'
                self.POPULATION_PLOTS_DIR = base_dir / 'figures' / 'population_plots'
                self.STATISTICAL_PLOTS_DIR = base_dir / 'figures' / 'statistical_plots'
                
                for dir_path in [self.INDIVIDUAL_PLOTS_DIR, self.POPULATION_PLOTS_DIR, self.STATISTICAL_PLOTS_DIR]:
                    dir_path.mkdir(parents=True, exist_ok=True)
                
                self.MOTOR_NOISE_THRESHOLD = 0.3
                self.SUCCESS_RATE_THRESHOLD = 0.68
                self.FIGURE_DPI = 300
        
        return MinimalConfig()

    # ==========================================================================
    # COMPREHENSIVE PLOTTING METHODS
    # ==========================================================================

    def generate_all_enhanced_plots(self, include_individual_plots: bool = False, 
                                  max_individual_subjects: int = None) -> Dict:
        """
        Generate ALL enhanced visualizations in one call.
        
        Parameters:
        -----------
        include_individual_plots : bool, default False
            Whether to generate individual participant plots (time-consuming)
        max_individual_subjects : int, optional
            Maximum number of subjects for individual plots (useful for testing)
            
        Returns:
        --------
        Dict : Summary of generated plots
        """
        
        print(f"🚀 Starting comprehensive enhanced plotting analysis...")
        print(f"⏰ Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"🎯 Individual plots: {'Enabled' if include_individual_plots else 'Disabled'}")
        print("=" * 80)
        
        results = {
            'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
            'individual_plots': 0,
            'population_plots': 0,
            'statistical_plots': 0,
            'generated_files': []
        }
        
        try:
            # 1. Individual stride change plots (OPTIONAL)
            if include_individual_plots:
                print("\n🎯 INDIVIDUAL STRIDE CHANGE PLOTS")
                print("-" * 50)
                individual_result = self.plot_all_individual_stride_changes(
                    trial_types=['vis1', 'invis', 'vis2'],
                    subject_ids=None,
                    save_summary=True,
                    max_subjects=max_individual_subjects
                )
                results['individual_plots'] = len(individual_result) if individual_result else 0
            else:
                print("\n⏭️ Skipping individual plots (set include_individual_plots=True to enable)")
            
            # 2. Enhanced population-level plots
            print("\n👥 ENHANCED POPULATION-LEVEL ANALYSIS PLOTS")
            print("-" * 50)
            population_result = self.generate_enhanced_population_plots()
            results['population_plots'] = population_result.get('n_plots', 0)
            results['generated_files'].extend(population_result.get('files', []))
            
            # 3. Statistical analysis plots
            print("\n📊 STATISTICAL ANALYSIS PLOTS")
            print("-" * 50)
            statistical_result = self.generate_statistical_plots()
            results['statistical_plots'] = statistical_result.get('n_plots', 0)
            results['generated_files'].extend(statistical_result.get('files', []))
            
            # 4. Summary dashboard
            print("\n📋 SUMMARY DASHBOARD")
            print("-" * 50)
            dashboard_result = self.generate_summary_dashboard()
            if dashboard_result.get('success'):
                results['generated_files'].append(dashboard_result['file'])
            
            # Final summary
            total_plots = results['individual_plots'] + results['population_plots'] + results['statistical_plots']
            print("\n" + "=" * 80)
            print("🎉 ENHANCED PLOTTING COMPLETED SUCCESSFULLY!")
            print("=" * 80)
            print(f"⏰ Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"📊 Total plots generated: {total_plots}")
            print(f"   • Individual plots: {results['individual_plots']}")
            print(f"   • Population plots: {results['population_plots']}")
            print(f"   • Statistical plots: {results['statistical_plots']}")
            print(f"📁 Files saved to: {self.config.BASE_OUTPUT_DIR}")
            
            if not include_individual_plots:
                print("💡 To generate individual plots, use: generate_all_enhanced_plots(include_individual_plots=True)")
            
            return results
            
        except Exception as e:
            print(f"\n❌ Enhanced plotting failed: {e}")
            results['error'] = str(e)
            raise

    def generate_enhanced_population_plots(self) -> Dict:
        """Generate ALL enhanced population-level analysis plots."""
        
        plot_count = 0
        generated_files = []
        df = self.filtered_df
        
        try:
            print("📊 Generating enhanced population-level analysis plots...")
            
            # 1. ENHANCED: Age vs Success Rates by trial/condition with motor noise coloring
            print("   🎯 Age vs Success Rates (Enhanced by trial/condition)...")
            self._plot_age_vs_success_rates_enhanced(df)
            plot_count += 1
            generated_files.append('age_vs_success_rates_enhanced.png')
            
            # 2. ENHANCED: Age vs Stride Variability by trial/condition with motor noise coloring
            print("   📏 Age vs Stride Variability (Enhanced by trial/condition)...")
            self._plot_age_vs_stride_variability_enhanced(df)
            plot_count += 1
            generated_files.append('age_vs_stride_variability_enhanced.png')
            
            # 3. Mean Stride Length vs Success Rate by trial/condition with motor noise coloring
            print("   📊 Mean Stride Length vs Success Rate (by trial/condition)...")
            self._plot_msl_vs_sr_enhanced(df)
            plot_count += 1
            generated_files.append('mean_stride_length_vs_success_rate.png')
            
            # 4. Age vs Mean Stride Length by trial/condition with motor noise coloring
            print("   📏 Age vs Mean Stride Length (Enhanced by trial/condition)...")
            self._plot_age_vs_mean_stride_length_enhanced(df)
            plot_count += 1
            generated_files.append('age_vs_mean_stride_length_enhanced.png')
            
            # 5. Age vs Mean Error by trial/condition with motor noise coloring
            print("   🎯 Age vs Mean Error (Enhanced by trial/condition)...")
            self._plot_age_vs_mean_error_enhanced(df)
            plot_count += 1
            generated_files.append('age_vs_mean_error_enhanced.png')
            
            # 6. NEW: Mean Error vs Success Rate by trial/condition with motor noise coloring
            print("   📊 Mean Error vs Success Rate (Enhanced by trial/condition)...")
            self._plot_mean_error_vs_success_rate_enhanced(df)
            plot_count += 1
            generated_files.append('mean_error_vs_success_rate_enhanced.png')
            
            # 7. Motor Noise vs Success Rate by trial/condition with age coloring
            print("   🧠 Motor Noise vs Success Rate (by trial/condition)...")
            self._plot_motor_noise_vs_success_rates_enhanced(df)
            plot_count += 1
            generated_files.append('motor_noise_vs_success_rates_enhanced.png')
            
            # 8. Age vs Motor Noise
            if 'age' in df.columns and 'mot_noise' in df.columns:
                print("   👥 Age vs Motor Noise...")
                self._plot_age_vs_motor_noise(df)
                plot_count += 1
                generated_files.append('age_vs_motor_noise.png')
            
            # 9. Correlation Matrix
            print("   🔗 Correlation Matrix...")
            self._plot_correlation_matrix(df)
            plot_count += 1
            generated_files.append('correlation_matrix.png')
            
            # 10. Success Rate Overview
            sr_cols = [col for col in df.columns if '_sr_' in col and '_const' in col]
            if sr_cols:
                print("   📊 Success Rate Overview...")
                self._plot_success_rate_overview(df, sr_cols)
                plot_count += 1
                generated_files.append('success_rate_overview.png')
            
            print(f"   ✅ Generated {plot_count} enhanced population-level plots")
            
            return {
                'n_plots': plot_count,
                'files': generated_files,
                'output_dir': str(self.population_plots_dir),
                'enhanced': True
            }
            
        except Exception as e:
            print(f"   ⚠️ Enhanced population plots failed: {e}")
            return {'n_plots': 0, 'error': str(e), 'enhanced': False}

    def generate_statistical_plots(self) -> Dict:
        """Generate statistical analysis visualization plots."""
        
        plot_count = 0
        generated_files = []
        
        try:
            print("📊 Generating statistical analysis plots...")
            
            # 1. Feature importance analysis (if available)
            if hasattr(self.analysis, 'analyzer'):
                print("   📈 Feature Importance Analysis...")
                self._plot_feature_importance_analysis()
                plot_count += 1
                generated_files.append('feature_importance_analysis.png')
            
            # 2. Age effects summary
            print("   👥 Age Effects Summary...")
            self._plot_age_effects_summary()
            plot_count += 1
            generated_files.append('age_effects_summary.png')
            
            # 3. Trial comparisons
            print("   🎮 Trial Type Comparisons...")
            self._plot_trial_comparisons()
            plot_count += 1
            generated_files.append('trial_comparisons.png')
            
            print(f"   ✅ Generated {plot_count} statistical analysis plots")
            
            return {
                'n_plots': plot_count,
                'files': generated_files,
                'output_dir': str(self.statistical_plots_dir)
            }
            
        except Exception as e:
            print(f"   ⚠️ Statistical plots failed: {e}")
            return {'n_plots': 0, 'error': str(e)}

    # ==========================================================================
    # ENHANCED POPULATION PLOTTING METHODS
    # ==========================================================================

    def _plot_age_vs_success_rates_enhanced(self, df: pd.DataFrame):
        """Enhanced age vs success rates by trial/condition with motor noise coloring."""
        
        sr_columns = [col for col in df.columns if '_sr_' in col and '_const' in col]
        if not sr_columns:
            print("   ⚠️ No success rate columns found")
            return
        
        trials = ['vis1', 'invis', 'vis2']
        conditions = ['max', 'min']
        has_motor_noise = 'mot_noise' in df.columns
        
        fig, axes = plt.subplots(len(trials), len(conditions), figsize=(12, 4*len(trials)))
        if len(trials) == 1:
            axes = axes.reshape(1, -1)
        
        for i, trial in enumerate(trials):
            for j, condition in enumerate(conditions):
                ax = axes[i, j]
                col = f'{trial}_sr_{condition}_const'
                
                if col in df.columns:
                    cols_to_use = ['age', col] + (['mot_noise'] if has_motor_noise else [])
                    valid_data = df[cols_to_use].dropna()
                    
                    if not valid_data.empty:
                        if has_motor_noise:
                            scatter = ax.scatter(valid_data['age'], valid_data[col], 
                                               c=valid_data['mot_noise'], cmap='plasma', 
                                               alpha=0.7, s=60, edgecolors='white', linewidths=0.5)
                            if i == 0 and j == len(conditions) - 1:
                                cbar = plt.colorbar(scatter, ax=ax)
                                cbar.set_label('Motor Noise', rotation=270, labelpad=15)
                        else:
                            ax.scatter(valid_data['age'], valid_data[col], alpha=0.7, s=60)
                        
                        # Add trend line and correlation
                        self._add_trendline(ax, valid_data['age'], valid_data[col])
                        
                        ax.set_xlabel('Age (years)')
                        ax.set_ylabel('Success Rate')
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                        ax.set_ylim(-0.05, 1.05)
                        ax.grid(True, alpha=0.3)
                    else:
                        ax.text(0.5, 0.5, 'No Data', ha='center', va='center',
                               transform=ax.transAxes, fontsize=12)
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                else:
                    ax.text(0.5, 0.5, 'Column Not Found', ha='center', va='center',
                           transform=ax.transAxes, fontsize=12)
                    ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
        
        plt.suptitle('Age vs Success Rates by Trial/Condition (Colored by Motor Noise)' if has_motor_noise 
                     else 'Age vs Success Rates by Trial/Condition', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'age_vs_success_rates_enhanced.png', 
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_age_vs_stride_variability_enhanced(self, df: pd.DataFrame):
        """Enhanced age vs stride variability by trial/condition with motor noise coloring."""
        
        sd_columns = [col for col in df.columns if '_sd_' in col and '_const' in col]
        if not sd_columns:
            print("   ⚠️ No stride variability columns found")
            return
        
        trials = ['vis1', 'invis', 'vis2']
        conditions = ['max', 'min']
        has_motor_noise = 'mot_noise' in df.columns
        
        fig, axes = plt.subplots(len(trials), len(conditions), figsize=(12, 4*len(trials)))
        if len(trials) == 1:
            axes = axes.reshape(1, -1)
        
        for i, trial in enumerate(trials):
            for j, condition in enumerate(conditions):
                ax = axes[i, j]
                col = f'{trial}_sd_{condition}_const'
                
                if col in df.columns:
                    cols_to_use = ['age', col] + (['mot_noise'] if has_motor_noise else [])
                    valid_data = df[cols_to_use].dropna()
                    
                    if not valid_data.empty:
                        if has_motor_noise:
                            scatter = ax.scatter(valid_data['age'], valid_data[col], 
                                               c=valid_data['mot_noise'], cmap='viridis', 
                                               alpha=0.7, s=60, edgecolors='white', linewidths=0.5)
                            if i == 0 and j == len(conditions) - 1:
                                cbar = plt.colorbar(scatter, ax=ax)
                                cbar.set_label('Motor Noise', rotation=270, labelpad=15)
                        else:
                            ax.scatter(valid_data['age'], valid_data[col], alpha=0.7, s=60)
                        
                        # Add trend line and correlation
                        self._add_trendline(ax, valid_data['age'], valid_data[col])
                        
                        ax.set_xlabel('Age (years)')
                        ax.set_ylabel('Stride Length SD')
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                        ax.grid(True, alpha=0.3)
                    else:
                        ax.text(0.5, 0.5, 'No Data', ha='center', va='center',
                               transform=ax.transAxes, fontsize=12)
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                else:
                    ax.text(0.5, 0.5, 'Column Not Found', ha='center', va='center',
                           transform=ax.transAxes, fontsize=12)
                    ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
        
        plt.suptitle('Age vs Stride Variability by Trial/Condition (Colored by Motor Noise)' if has_motor_noise 
                     else 'Age vs Stride Variability by Trial/Condition', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'age_vs_stride_variability_enhanced.png', 
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_msl_vs_sr_enhanced(self, df: pd.DataFrame):
        """Mean stride length vs success rate by trial/condition with motor noise coloring."""
        
        msl_columns = [col for col in df.columns if '_msl_' in col and '_const' in col]
        sr_columns = [col for col in df.columns if '_sr_' in col and '_const' in col]
        
        if not msl_columns or not sr_columns:
            print("   ⚠️ Missing mean stride length or success rate columns")
            return
        
        trials = ['vis1', 'invis', 'vis2']
        conditions = ['max', 'min']
        has_motor_noise = 'mot_noise' in df.columns
        
        fig, axes = plt.subplots(len(trials), len(conditions), figsize=(12, 4*len(trials)))
        if len(trials) == 1:
            axes = axes.reshape(1, -1)
        
        for i, trial in enumerate(trials):
            for j, condition in enumerate(conditions):
                ax = axes[i, j]
                msl_col = f'{trial}_msl_{condition}_const'
                sr_col = f'{trial}_sr_{condition}_const'
                
                if msl_col in df.columns and sr_col in df.columns:
                    cols_to_use = [msl_col, sr_col] + (['mot_noise'] if has_motor_noise else [])
                    valid_data = df[cols_to_use].dropna()
                    
                    if not valid_data.empty:
                        if has_motor_noise:
                            scatter = ax.scatter(valid_data[msl_col], valid_data[sr_col], 
                                               c=valid_data['mot_noise'], cmap='coolwarm', 
                                               alpha=0.7, s=60, edgecolors='white', linewidths=0.5)
                            if i == 0 and j == len(conditions) - 1:
                                cbar = plt.colorbar(scatter, ax=ax)
                                cbar.set_label('Motor Noise', rotation=270, labelpad=15)
                        else:
                            ax.scatter(valid_data[msl_col], valid_data[sr_col], alpha=0.7, s=60)
                        
                        # Add trend line and correlation
                        self._add_trendline(ax, valid_data[msl_col], valid_data[sr_col])
                        
                        ax.set_xlabel('Mean Stride Length')
                        ax.set_ylabel('Success Rate')
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                        ax.set_ylim(-0.05, 1.05)
                        ax.grid(True, alpha=0.3)
                    else:
                        ax.text(0.5, 0.5, 'No Data', ha='center', va='center',
                               transform=ax.transAxes, fontsize=12)
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                else:
                    ax.text(0.5, 0.5, 'Columns Not Found', ha='center', va='center',
                           transform=ax.transAxes, fontsize=12)
                    ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
        
        plt.suptitle('Mean Stride Length vs Success Rate by Trial/Condition (Colored by Motor Noise)' if has_motor_noise 
                     else 'Mean Stride Length vs Success Rate by Trial/Condition', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'mean_stride_length_vs_success_rate.png', 
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_age_vs_mean_stride_length_enhanced(self, df: pd.DataFrame):
        """Enhanced age vs mean stride length by trial/condition with motor noise coloring."""
        
        msl_columns = [col for col in df.columns if '_msl_' in col and '_const' in col]
        if not msl_columns:
            print("   ⚠️ No mean stride length columns found")
            return
        
        trials = ['vis1', 'invis', 'vis2']
        conditions = ['max', 'min']
        has_motor_noise = 'mot_noise' in df.columns
        
        fig, axes = plt.subplots(len(trials), len(conditions), figsize=(12, 4*len(trials)))
        if len(trials) == 1:
            axes = axes.reshape(1, -1)
        
        for i, trial in enumerate(trials):
            for j, condition in enumerate(conditions):
                ax = axes[i, j]
                col = f'{trial}_msl_{condition}_const'
                
                if col in df.columns:
                    cols_to_use = ['age', col] + (['mot_noise'] if has_motor_noise else [])
                    valid_data = df[cols_to_use].dropna()
                    
                    if not valid_data.empty:
                        if has_motor_noise:
                            scatter = ax.scatter(valid_data['age'], valid_data[col], 
                                               c=valid_data['mot_noise'], cmap='plasma', 
                                               alpha=0.7, s=60, edgecolors='white', linewidths=0.5)
                            if i == 0 and j == len(conditions) - 1:
                                cbar = plt.colorbar(scatter, ax=ax)
                                cbar.set_label('Motor Noise', rotation=270, labelpad=15)
                        else:
                            trial_colors = {'vis1': '#1f77b4', 'invis': '#ff7f0e', 'vis2': '#2ca02c'}
                            ax.scatter(valid_data['age'], valid_data[col], 
                                     alpha=0.7, s=60, color=trial_colors.get(trial, 'steelblue'),
                                     edgecolors='white', linewidths=0.5)
                        
                        # Add trend line and correlation
                        self._add_trendline(ax, valid_data['age'], valid_data[col])
                        
                        ax.set_xlabel('Age (years)')
                        ax.set_ylabel('Mean Stride Length')
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                        ax.grid(True, alpha=0.3)
                    else:
                        ax.text(0.5, 0.5, 'No Data', ha='center', va='center',
                               transform=ax.transAxes, fontsize=12)
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                else:
                    ax.text(0.5, 0.5, 'Column Not Found', ha='center', va='center',
                           transform=ax.transAxes, fontsize=12)
                    ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
        
        plt.suptitle('Age vs Mean Stride Length by Trial/Condition (Colored by Motor Noise)' if has_motor_noise 
                     else 'Age vs Mean Stride Length by Trial/Condition', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'age_vs_mean_stride_length_enhanced.png', 
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_age_vs_mean_error_enhanced(self, df: pd.DataFrame):
        """Enhanced age vs mean error by trial/condition with motor noise coloring."""
        
        error_columns = [col for col in df.columns if '_error_' in col and '_const' in col]
        if not error_columns:
            print("   ⚠️ No mean error columns found")
            return
        
        trials = ['vis1', 'invis', 'vis2']
        conditions = ['max', 'min']
        has_motor_noise = 'mot_noise' in df.columns
        
        fig, axes = plt.subplots(len(trials), len(conditions), figsize=(12, 4*len(trials)))
        if len(trials) == 1:
            axes = axes.reshape(1, -1)
        
        for i, trial in enumerate(trials):
            for j, condition in enumerate(conditions):
                ax = axes[i, j]
                col = f'{trial}_error_{condition}_const'
                
                if col in df.columns:
                    cols_to_use = ['age', col] + (['mot_noise'] if has_motor_noise else [])
                    valid_data = df[cols_to_use].dropna()
                    
                    if not valid_data.empty:
                        if has_motor_noise:
                            scatter = ax.scatter(valid_data['age'], valid_data[col], 
                                               c=valid_data['mot_noise'], alpha=0.7, s=60,
                                               edgecolors='white', linewidths=0.5)
                            if i == 0 and j == len(conditions) - 1:
                                cbar = plt.colorbar(scatter, ax=ax)
                                cbar.set_label('Motor Noise', rotation=270, labelpad=15)
                        else:
                            trial_colors = {'vis1': '#1f77b4', 'invis': '#ff7f0e', 'vis2': '#2ca02c'}
                            ax.scatter(valid_data['age'], valid_data[col], 
                                     alpha=0.7, s=60, color=trial_colors.get(trial, 'steelblue'),
                                     edgecolors='white', linewidths=0.5)
                        
                        # Add trend line and correlation
                        self._add_trendline(ax, valid_data['age'], valid_data[col])
                        
                        # Add horizontal line at zero (perfect accuracy)
                        ax.axhline(y=0, color='green', linestyle='--', alpha=0.5, 
                                  label='Perfect Accuracy (Error = 0)')
                        
                        ax.set_xlabel('Age (years)')
                        ax.set_ylabel('Mean Error (Stride Length - Target)')
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                        ax.grid(True, alpha=0.3)
                        ax.set_ylim(-0.5, 0.5)
                        
                        # Add legend only to first subplot to avoid clutter
                        if i == 0 and j == 0:
                            ax.legend(loc='upper right', fontsize=9)
                    else:
                        ax.text(0.5, 0.5, 'No Data', ha='center', va='center',
                               transform=ax.transAxes, fontsize=12)
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                else:
                    ax.text(0.5, 0.5, 'Column Not Found', ha='center', va='center',
                           transform=ax.transAxes, fontsize=12)
                    ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
        
        plt.suptitle('Age vs Mean Error by Trial/Condition (Colored by Motor Noise)' if has_motor_noise 
                     else 'Age vs Mean Error by Trial/Condition', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'age_vs_mean_error_enhanced.png', 
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_mean_error_vs_success_rate_enhanced(self, df: pd.DataFrame):
        """Enhanced mean error vs success rate by trial/condition with motor noise coloring."""
        
        error_columns = [col for col in df.columns if '_error_' in col and '_const' in col]
        sr_columns = [col for col in df.columns if '_sr_' in col and '_const' in col]
        
        if not error_columns or not sr_columns:
            print("   ⚠️ Missing error or success rate columns")
            return
        
        trials = ['vis1', 'invis', 'vis2']
        conditions = ['max', 'min']
        has_motor_noise = 'mot_noise' in df.columns
        has_age = 'age' in df.columns
        
        fig, axes = plt.subplots(len(trials), len(conditions), figsize=(12, 4*len(trials)))
        if len(trials) == 1:
            axes = axes.reshape(1, -1)
        
        for i, trial in enumerate(trials):
            for j, condition in enumerate(conditions):
                ax = axes[i, j]
                error_col = f'{trial}_error_{condition}_const'
                sr_col = f'{trial}_sr_{condition}_const'
                
                if error_col in df.columns and sr_col in df.columns:
                    cols_to_use = [error_col, sr_col] + (['mot_noise'] if has_motor_noise else []) + (['age'] if has_age else [])
                    valid_data = df[cols_to_use].dropna()
                    
                    if not valid_data.empty:
                        if has_motor_noise:
                            scatter = ax.scatter(valid_data[error_col], valid_data[sr_col], 
                                               c=valid_data['mot_noise'], cmap='plasma', 
                                               alpha=0.7, s=60, edgecolors='white', linewidths=0.5)
                            if i == 0 and j == len(conditions) - 1:
                                cbar = plt.colorbar(scatter, ax=ax)
                                cbar.set_label('Motor Noise', rotation=270, labelpad=15)
                        elif has_age:
                            scatter = ax.scatter(valid_data[error_col], valid_data[sr_col], 
                                               c=valid_data['age'], cmap='viridis', 
                                               alpha=0.7, s=60, edgecolors='white', linewidths=0.5)
                            if i == 0 and j == len(conditions) - 1:
                                cbar = plt.colorbar(scatter, ax=ax)
                                cbar.set_label('Age (years)', rotation=270, labelpad=15)
                        else:
                            trial_colors = {'vis1': '#1f77b4', 'invis': '#ff7f0e', 'vis2': '#2ca02c'}
                            ax.scatter(valid_data[error_col], valid_data[sr_col], 
                                     alpha=0.7, s=60, color=trial_colors.get(trial, 'steelblue'),
                                     edgecolors='white', linewidths=0.5)
                        
                        # Add trend line and correlation
                        self._add_trendline(ax, valid_data[error_col], valid_data[sr_col])
                        
                        # Add reference lines
                        ax.axvline(x=0, color='green', linestyle='--', alpha=0.5, 
                                  label='Perfect Accuracy (Error = 0)')
                        ax.axhline(y=0.68, color='red', linestyle='--', alpha=0.5, 
                                  label='68% Success Threshold')
                        
                        ax.set_xlabel('Mean Error (Stride Length - Target)')
                        ax.set_ylabel('Success Rate')
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                        ax.set_ylim(-0.05, 1.05)
                        ax.set_xlim(-0.5, 0.5)
                        ax.grid(True, alpha=0.3)
                        
                        # Add legend only to first subplot to avoid clutter
                        if i == 0 and j == 0:
                            ax.legend(loc='upper right', fontsize=9)
                    else:
                        ax.text(0.5, 0.5, 'No Data', ha='center', va='center',
                               transform=ax.transAxes, fontsize=12)
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                else:
                    ax.text(0.5, 0.5, 'Columns Not Found', ha='center', va='center',
                           transform=ax.transAxes, fontsize=12)
                    ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
        
        color_label = 'Motor Noise' if has_motor_noise else ('Age' if has_age else '')
        title_suffix = f' (Colored by {color_label})' if color_label else ''
        
        plt.suptitle(f'Mean Error vs Success Rate by Trial/Condition{title_suffix}', 
                     fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'mean_error_vs_success_rate_enhanced.png', 
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_motor_noise_vs_success_rates_enhanced(self, df: pd.DataFrame):
        """Motor noise vs success rates by trial/condition with age coloring."""
        
        if 'mot_noise' not in df.columns:
            print("   ⚠️ No motor noise data available")
            return
        
        sr_columns = [col for col in df.columns if '_sr_' in col and '_const' in col]
        if not sr_columns:
            print("   ⚠️ No success rate columns found")
            return
        
        trials = ['vis1', 'invis', 'vis2']
        conditions = ['max', 'min']
        has_age = 'age' in df.columns
        
        fig, axes = plt.subplots(len(trials), len(conditions), figsize=(12, 4*len(trials)))
        if len(trials) == 1:
            axes = axes.reshape(1, -1)
        
        for i, trial in enumerate(trials):
            for j, condition in enumerate(conditions):
                ax = axes[i, j]
                col = f'{trial}_sr_{condition}_const'
                
                if col in df.columns:
                    cols_to_use = ['mot_noise', col] + (['age'] if has_age else [])
                    valid_data = df[cols_to_use].dropna()
                    
                    if not valid_data.empty:
                        if has_age:
                            scatter = ax.scatter(valid_data['mot_noise'], valid_data[col], 
                                               c=valid_data['age'], alpha=0.7, s=60, 
                                               edgecolors='white', linewidths=0.5)
                            if i == 0 and j == len(conditions) - 1:
                                cbar = plt.colorbar(scatter, ax=ax)
                                cbar.set_label('Age (years)', rotation=270, labelpad=15)
                        else:
                            ax.scatter(valid_data['mot_noise'], valid_data[col], 
                                     alpha=0.7, s=60, color='steelblue',
                                     edgecolors='white', linewidths=0.5)
                        
                        # Add threshold lines
                        motor_noise_threshold = getattr(self.config, 'MOTOR_NOISE_THRESHOLD', 0.3)
                        success_threshold = getattr(self.config, 'SUCCESS_RATE_THRESHOLD', 0.68)
                        
                        ax.axvline(x=motor_noise_threshold, color='red', 
                                  linestyle='--', alpha=0.7, linewidth=2,
                                  label=f'Motor Noise Threshold')
                        ax.axhline(y=success_threshold, color='green', 
                                  linestyle='--', alpha=0.7, linewidth=2,
                                  label=f'Success Threshold')
                        
                        ax.set_xlabel('Motor Noise')
                        ax.set_ylabel('Success Rate')
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                        ax.set_ylim(-0.05, 1.05)
                        ax.grid(True, alpha=0.3)
                        
                        # Add legend only to the first subplot to avoid clutter
                        if i == 0 and j == 0:
                            ax.legend(loc='upper right', fontsize=9)
                    else:
                        ax.text(0.5, 0.5, 'No Data', ha='center', va='center',
                               transform=ax.transAxes, fontsize=12)
                        ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
                else:
                    ax.text(0.5, 0.5, 'Column Not Found', ha='center', va='center',
                           transform=ax.transAxes, fontsize=12)
                    ax.set_title(f'{trial.upper()}: {condition.capitalize()} Target')
        
        plt.suptitle('Motor Noise vs Success Rates by Trial/Condition (Colored by Age)' if has_age 
                     else 'Motor Noise vs Success Rates by Trial/Condition', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'motor_noise_vs_success_rates_enhanced.png', 
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_age_vs_motor_noise(self, df: pd.DataFrame):
        """Plot age vs motor noise relationship with robust error handling."""
        
        if 'age' not in df.columns or 'mot_noise' not in df.columns:
            print("   ⚠️ Missing age or motor noise columns")
            return
        
        plt.figure(figsize=(10, 8))
        
        try:
            # Convert to numeric and drop NaN values
            age_data = pd.to_numeric(df['age'], errors='coerce')
            motor_noise_data = pd.to_numeric(df['mot_noise'], errors='coerce')
            
            plot_data = pd.DataFrame({
                'age': age_data,
                'mot_noise': motor_noise_data
            }).dropna()
            
            if plot_data.empty or len(plot_data) < 2:
                plt.text(0.5, 0.5, 'Insufficient valid data', ha='center', va='center',
                        transform=plt.gca().transAxes, fontsize=14)
                plt.title('Age vs Motor Noise - Insufficient Data')
            else:
                # Create scatter plot
                plt.scatter(plot_data['age'], plot_data['mot_noise'], 
                           alpha=0.6, s=60, color='steelblue', 
                           edgecolors='white', linewidths=0.5)
                
                # Add trend line and correlation
                self._add_trendline(plt.gca(), plot_data['age'], plot_data['mot_noise'])
                
                # Add motor noise threshold line
                motor_noise_threshold = getattr(self.config, 'MOTOR_NOISE_THRESHOLD', 0.3)
                plt.axhline(y=motor_noise_threshold, color='red', linestyle='--', alpha=0.7, 
                           label=f'Threshold ({motor_noise_threshold})')
                
                plt.xlabel('Age (years)', fontsize=12)
                plt.ylabel('Motor Noise', fontsize=12)
                plt.title('Age vs Motor Noise Relationship', fontsize=14, fontweight='bold')
                plt.grid(True, alpha=0.3)
                plt.legend()
                
                # Set reasonable axis limits
                age_range = plot_data['age'].max() - plot_data['age'].min()
                motor_range = plot_data['mot_noise'].max() - plot_data['mot_noise'].min()
                
                plt.xlim(plot_data['age'].min() - age_range*0.05, 
                        plot_data['age'].max() + age_range*0.05)
                plt.ylim(max(0, plot_data['mot_noise'].min() - motor_range*0.05), 
                        plot_data['mot_noise'].max() + motor_range*0.05)
        
        except Exception as e:
            print(f"   ⚠️ Error in age vs motor noise plot: {e}")
            plt.text(0.5, 0.5, f'Error creating plot:\\n{str(e)}', 
                    ha='center', va='center', transform=plt.gca().transAxes, 
                    fontsize=12, style='italic')
            plt.title('Age vs Motor Noise - Error')
        
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'age_vs_motor_noise.png', 
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_correlation_matrix(self, df: pd.DataFrame):
        """Plot correlation matrix of key variables."""
        
        key_cols = ['age']
        if 'mot_noise' in df.columns:
            key_cols.append('mot_noise')
        if 'pref_asymmetry' in df.columns:
            key_cols.append('pref_asymmetry')
        
        # Add success rate columns
        sr_cols = [col for col in df.columns if '_sr_' in col and '_const' in col]
        key_cols.extend(sr_cols[:6])  # Limit to prevent overcrowding
        
        if len(key_cols) > 1:
            plt.figure(figsize=(12, 10))
            
            corr_matrix = df[key_cols].corr()
            mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
            
            sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
                       square=True, linewidths=0.5, fmt='.2f')
            
            plt.title('Correlation Matrix of Key Variables', fontsize=16, fontweight='bold')
            plt.tight_layout()
            
            plt.savefig(self.population_plots_dir / 'correlation_matrix.png',
                       dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
            plt.close()

    def _plot_success_rate_overview(self, df: pd.DataFrame, sr_cols: List[str]):
        """Plot success rate overview across conditions."""
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # 1. Mean success rates by condition
        sr_means = [df[col].mean() for col in sr_cols[:8]]  # Limit for readability
        sr_names = [col.replace('_sr_', ' ').replace('_const', '') for col in sr_cols[:8]]
        
        bars = ax1.bar(range(len(sr_names)), sr_means, alpha=0.7, color='steelblue')
        ax1.set_title('Mean Success Rates by Condition')
        ax1.set_ylabel('Success Rate')
        ax1.set_ylim(0, 1)
        ax1.set_xticks(range(len(sr_names)))
        ax1.set_xticklabels(sr_names, rotation=45, ha='right')
        ax1.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.2f}', ha='center', va='bottom')
        
        # 2. Success rate distributions
        plot_data = []
        for col in sr_cols[:6]:  # Limit for readability
            trial_type = col.split('_')[0]
            condition = col.split('_')[2]
            values = df[col].dropna()
            for val in values:
                plot_data.append({
                    'condition': f'{trial_type}_{condition}',
                    'success_rate': val
                })
        
        if plot_data:
            plot_df = pd.DataFrame(plot_data)
            sns.boxplot(data=plot_df, x='condition', y='success_rate', ax=ax2)
            ax2.set_title('Success Rate Distributions')
            ax2.set_ylabel('Success Rate')
            ax2.set_ylim(0, 1)
            plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
        
        plt.tight_layout()
        plt.savefig(self.population_plots_dir / 'success_rate_overview.png',
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    # ==========================================================================
    # STATISTICAL ANALYSIS PLOTTING METHODS
    # ==========================================================================

    def _plot_feature_importance_analysis(self):
        """Plot feature importance analysis if regression results are available."""
        
        if not hasattr(self.analysis, 'analyzer'):
            print("   ⚠️ No analyzer available for feature importance")
            return
        
        try:
            # Try to run a quick regression analysis
            regression_results = self.analysis.analyzer.run_regression_analysis()
            
            if 'feature_importances' in regression_results:
                plt.figure(figsize=(10, 6))
                
                features = list(regression_results['feature_importances'].keys())
                importances = list(regression_results['feature_importances'].values())
                
                bars = plt.bar(features, importances, alpha=0.7, color='steelblue')
                plt.title('Feature Importance for Success Rate Prediction')
                plt.ylabel('Importance')
                plt.xticks(rotation=45, ha='right')
                plt.grid(True, alpha=0.3)
                
                # Add value labels
                for bar, importance in zip(bars, importances):
                    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
                            f'{importance:.3f}', ha='center', va='bottom')
                
                plt.tight_layout()
                plt.savefig(self.statistical_plots_dir / 'feature_importance_analysis.png',
                           dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
                plt.close()
            
        except Exception as e:
            print(f"   ⚠️ Feature importance analysis failed: {e}")

    def _plot_age_effects_summary(self):
        """Plot summary of age effects on performance measures."""
        
        df = self.filtered_df
        if 'age' not in df.columns:
            print("   ⚠️ No age data available for age effects analysis")
            return
        
        # Calculate age effects for success rate measures
        sr_cols = [col for col in df.columns if '_sr_' in col and '_const' in col]
        age_effects = []
        
        for col in sr_cols[:10]:  # Limit for readability
            valid_data = df[['age', col]].dropna()
            if len(valid_data) > 10:
                try:
                    r, p = pearsonr(valid_data['age'], valid_data[col])
                    age_effects.append({
                        'measure': col.replace('_sr_', ' ').replace('_const', ''),
                        'correlation': r,
                        'p_value': p,
                        'significant': p < 0.05
                    })
                except:
                    continue
        
        if not age_effects:
            print("   ⚠️ No valid age effects calculated")
            return
        
        # Sort by correlation strength
        age_effects.sort(key=lambda x: abs(x['correlation']), reverse=True)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
        
        # Correlation plot
        measures = [e['measure'] for e in age_effects]
        correlations = [e['correlation'] for e in age_effects]
        significant = [e['significant'] for e in age_effects]
        
        colors = ['red' if sig else 'blue' for sig in significant]
        bars = ax1.barh(range(len(measures)), correlations, color=colors, alpha=0.7)
        ax1.set_yticks(range(len(measures)))
        ax1.set_yticklabels([m.replace('_', ' ') for m in measures])
        ax1.set_xlabel('Correlation with Age')
        ax1.set_title('Age Effect Sizes')
        ax1.axvline(x=0, color='black', linestyle='-', alpha=0.3)
        ax1.grid(True, alpha=0.3)
        
        # Add legend
        red_patch = plt.Rectangle((0,0),1,1, facecolor='red', alpha=0.7, label='Significant (p < 0.05)')
        blue_patch = plt.Rectangle((0,0),1,1, facecolor='blue', alpha=0.7, label='Not significant')
        ax1.legend(handles=[red_patch, blue_patch])
        
        # P-value plot
        p_values = [e['p_value'] for e in age_effects]
        log_p = [-np.log10(p) for p in p_values]
        bars2 = ax2.barh(range(len(measures)), log_p, color=colors, alpha=0.7)
        ax2.set_yticks(range(len(measures)))
        ax2.set_yticklabels([m.replace('_', ' ') for m in measures])
        ax2.set_xlabel('-log10(p-value)')
        ax2.set_title('Statistical Significance')
        ax2.axvline(x=-np.log10(0.05), color='red', linestyle='--', alpha=0.8, label='p = 0.05')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        plt.savefig(self.statistical_plots_dir / 'age_effects_summary.png',
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def _plot_trial_comparisons(self):
        """Plot comparisons between trial types."""
        
        df = self.filtered_df
        
        # Compare success rates across trial types
        comparison_data = []
        
        for condition in ['max', 'min']:
            for trial in ['vis1', 'invis', 'vis2']:
                col = f'{trial}_sr_{condition}_const'
                if col in df.columns:
                    values = df[col].dropna()
                    for val in values:
                        comparison_data.append({
                            'trial_type': trial,
                            'condition': condition,
                            'success_rate': val
                        })
        
        if not comparison_data:
            print("   ⚠️ No trial comparison data available")
            return
        
        comparison_df = pd.DataFrame(comparison_data)
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. Box plot by trial type
        sns.boxplot(data=comparison_df, x='trial_type', y='success_rate', ax=axes[0, 0])
        axes[0, 0].set_title('Success Rates by Trial Type')
        axes[0, 0].set_ylabel('Success Rate')
        axes[0, 0].set_ylim(-0.05, 1.05)
        
        # 2. Box plot by condition
        sns.boxplot(data=comparison_df, x='condition', y='success_rate', ax=axes[0, 1])
        axes[0, 1].set_title('Success Rates by Target Condition')
        axes[0, 1].set_ylabel('Success Rate')
        axes[0, 1].set_ylim(-0.05, 1.05)
        
        # 3. Box plot by trial type and condition
        sns.boxplot(data=comparison_df, x='trial_type', y='success_rate', 
                   hue='condition', ax=axes[1, 0])
        axes[1, 0].set_title('Success Rates by Trial Type and Condition')
        axes[1, 0].set_ylabel('Success Rate')
        axes[1, 0].set_ylim(-0.05, 1.05)
        
        # 4. Violin plot for distribution shapes
        sns.violinplot(data=comparison_df, x='trial_type', y='success_rate', 
                      hue='condition', ax=axes[1, 1])
        axes[1, 1].set_title('Success Rate Distributions')
        axes[1, 1].set_ylabel('Success Rate')
        axes[1, 1].set_ylim(-0.05, 1.05)
        
        plt.suptitle('Trial Type Comparisons Across Conditions', fontsize=16, fontweight='bold')
        plt.tight_layout()
        
        plt.savefig(self.statistical_plots_dir / 'trial_comparisons.png',
                   dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        plt.close()

    def generate_summary_dashboard(self) -> Dict:
        """Generate comprehensive summary dashboard."""
        
        try:
            df = self.filtered_df
            
            fig = plt.figure(figsize=(16, 12))
            gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
            
            # 1. Age distribution
            ax1 = fig.add_subplot(gs[0, 0])
            if 'age' in df.columns:
                df['age'].hist(bins=range(7, 19), ax=ax1, alpha=0.7, color='skyblue')
                ax1.set_title('Age Distribution')
                ax1.set_xlabel('Age (years)')
                ax1.set_ylabel('Count')
            
            # 2. Motor noise distribution
            ax2 = fig.add_subplot(gs[0, 1])
            if 'mot_noise' in df.columns:
                df['mot_noise'].hist(bins=15, ax=ax2, alpha=0.7, color='lightcoral')
                motor_noise_threshold = getattr(self.config, 'MOTOR_NOISE_THRESHOLD', 0.3)
                ax2.axvline(motor_noise_threshold, color='red', linestyle='--', 
                           label=f"Threshold ({motor_noise_threshold})")
                ax2.set_title('Motor Noise Distribution')
                ax2.set_xlabel('Motor Noise')
                ax2.set_ylabel('Count')
                ax2.legend()
            
            # 3. Success rates overview
            ax3 = fig.add_subplot(gs[0, 2])
            sr_cols = [col for col in df.columns if '_sr_' in col and '_const' in col]
            if sr_cols:
                sr_means = [df[col].mean() for col in sr_cols[:6]]
                sr_names = [col.replace('_sr_', ' ').replace('_const', '') for col in sr_cols[:6]]
                
                bars = ax3.bar(range(len(sr_names)), sr_means, alpha=0.7)
                ax3.set_title('Mean Success Rates')
                ax3.set_ylabel('Success Rate')
                ax3.set_ylim(0, 1)
                ax3.set_xticks(range(len(sr_names)))
                ax3.set_xticklabels(sr_names, rotation=45, ha='right')
            
            # 4. Summary statistics table
            ax4 = fig.add_subplot(gs[1:, :])
            ax4.axis('tight')
            ax4.axis('off')
            
            # Create summary table
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            motor_noise_threshold = getattr(self.config, 'MOTOR_NOISE_THRESHOLD', 0.3)
            
            summary_data = [
                ['Total Subjects', f"{len(df)}"],
                ['Age Range', f"{df['age'].min():.1f} - {df['age'].max():.1f} years" if 'age' in df.columns else "N/A"],
                ['Mean Age ± SD', f"{df['age'].mean():.1f} ± {df['age'].std():.1f} years" if 'age' in df.columns else "N/A"],
                ['Motor Noise Range', f"{df['mot_noise'].min():.3f} - {df['mot_noise'].max():.3f}" if 'mot_noise' in df.columns else "N/A"],
                ['Success Rate Measures', f"{len(sr_cols)}"],
                ['High Motor Noise Subjects', f"{(df['mot_noise'] > motor_noise_threshold).sum()}" if 'mot_noise' in df.columns else "N/A"],
                ['Enhanced Population Plots', "✅ Age vs SR, Age vs SD, MSL vs SR (by trial/condition)"],
                ['Motor Noise Coloring', "✅ All plots colored by motor noise level"],
                ['Analysis Timestamp', timestamp]
            ]
            
            table = ax4.table(cellText=summary_data, colLabels=['Metric', 'Value'],
                             cellLoc='left', loc='center')
            table.auto_set_font_size(False)
            table.set_fontsize(12)
            table.scale(1.2, 2)
            ax4.set_title('Enhanced Motor Learning Analysis Summary', fontsize=16, fontweight='bold', pad=20)
            
            plt.suptitle('Enhanced Motor Learning Analysis Dashboard', fontsize=18, y=0.95)
            
            # Save dashboard
            dashboard_file = self.statistical_plots_dir / f'analysis_dashboard_{timestamp}.png'
            plt.savefig(dashboard_file, dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
            plt.close()
            
            print(f"   Generated summary dashboard: {dashboard_file.name}")
            
            return {
                'success': True,
                'file': str(dashboard_file)
            }
            
        except Exception as e:
            print(f"   ⚠️ Dashboard generation failed: {e}")
            return {'success': False, 'error': str(e)}

    # ==========================================================================
    # INDIVIDUAL PLOTTING METHODS (EXISTING)
    # ==========================================================================

    def plot_all_individual_stride_changes(self, trial_types: List[str] = None, 
                                         subject_ids: List[str] = None,
                                         save_summary: bool = True,
                                         max_subjects: int = None) -> Dict:
        """Generate stride change distribution plots for all participants."""
        
        if trial_types is None:
            trial_types = ['vis1', 'invis', 'vis2']
        
        if subject_ids is None:
            subject_ids = list(self.data_manager.processed_data.keys())
        
        # Limit subjects if requested
        if max_subjects and len(subject_ids) > max_subjects:
            subject_ids = subject_ids[:max_subjects]
            print(f"🔄 Limited to first {max_subjects} subjects for testing")
        
        print(f"🎯 Generating individual stride change plots...")
        print(f"   📊 {len(subject_ids)} subjects")
        print(f"   🎮 Trial types: {trial_types}")
        print(f"   💾 Saving to: {self.individual_plots_dir}")
        
        all_stats = {}
        successful_plots = 0
        failed_plots = 0
        
        # Progress bar for subjects
        for subject_id in tqdm(subject_ids, desc="Processing subjects"):
            subject_stats = {}
            
            # Get subject metadata
            if subject_id in self.data_manager.processed_data:
                age = self.data_manager.processed_data[subject_id]['metadata'].get('age_months', np.nan) / 12
                subject_stats['age'] = age
            
            # Process each trial type
            for trial_type in trial_types:
                try:
                    # Use our own implementation
                    stats = self._plot_individual_stride_change_internal(
                        subject_id, trial_type, save=True, show_stats=False
                    )
                    
                    if stats:
                        subject_stats[trial_type] = stats
                        successful_plots += 1
                    else:
                        failed_plots += 1
                        
                except Exception as e:
                    print(f"   ⚠️ Error plotting {subject_id} {trial_type}: {str(e)}")
                    failed_plots += 1
                    continue
            
            if subject_stats:
                all_stats[subject_id] = subject_stats
        
        print(f"✅ Completed: {successful_plots} successful plots, {failed_plots} failed")
        
        # Save summary if requested
        if save_summary:
            self._save_stride_analysis_summary(all_stats)
        
        return all_stats

    def _plot_individual_stride_change_internal(self, subject_id: str, trial_types: List[str] = None,
                                               stride_col: str = 'Sum of gains and steps',
                                               figsize: Tuple[int, int] = (16, 18), 
                                               save: bool = True, alpha: float = 0.7,
                                               show_stats: bool = False) -> Optional[Dict]:
        """
        Internal method for plotting individual stride changes with FIXED 3x2 grid layout.
        Always creates exactly 6 subplots (3 trials × 2 conditions) regardless of data availability.
        """
        
        if trial_types is None:
            trial_types = ['vis1', 'invis', 'vis2']
        
        # Check if subject exists
        if subject_id not in self.data_manager.processed_data:
            return None
        
        # Get subject age for title
        subject_age = self.data_manager.processed_data[subject_id]['metadata'].get('age_months', np.nan) / 12
        
        # FIXED: Always create 3x2 grid regardless of data availability
        fig, axes = plt.subplots(3, 2, figsize=figsize, sharey=False)
        
        # Define the fixed layout mapping
        layout_mapping = {
            'vis1': (0, 0, 0, 1),    # Row 0, columns 0 and 1
            'invis': (1, 0, 1, 1),   # Row 1, columns 0 and 1  
            'vis2': (2, 0, 2, 1)     # Row 2, columns 0 and 1
        }
        
        # Condition names for column headers
        condition_names = ['Upper Target (Max Constant)', 'Lower Target (Min Constant)']
        
        # Helper function to get period data
        def get_period_data(df, constant_condition, length=20):
            if df is None or df.empty:
                return None
                
            min_target = df['Target size'].min()
            target_tolerance = 0.001
            
            min_target_periods = df[df['Target size'] <= min_target + target_tolerance]
            if min_target_periods.empty:
                return None
                
            const_value = (
                min_target_periods['Constant'].max() if constant_condition == 'max'
                else min_target_periods['Constant'].min()
            )
            
            period_data = min_target_periods[
                np.isclose(min_target_periods['Constant'], const_value, rtol=1e-5)
            ]
            
            if period_data.empty:
                return None
                
            return period_data.tail(length)
        
        # Function to process and plot data for each condition
        def plot_condition_data(ax, period_data, condition_name, const_type, trial_type, row, col):
            # Set title and labels regardless of data availability
            ax.set_title(f'{trial_type.upper()}: {condition_name}\\n(last 20 strides)', 
                        fontsize=12, fontweight='bold')
            ax.set_xlabel(f'Change in {stride_col.replace("_", " ")}', fontsize=11)
            ax.set_ylabel('Probability Density', fontsize=11)
            ax.grid(True, alpha=0.3)
            
            if period_data is None or period_data.empty:
                ax.text(0.5, 0.5, f'No {condition_name.split()[0].lower()}\\ntarget data available', 
                       ha='center', va='center', transform=ax.transAxes,
                       fontsize=12, style='italic', color='gray',
                       bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.5))
                ax.set_xlim(-2, 2)
                ax.set_ylim(0, 1)
                return None
            
            # Sort by stride number and calculate stride length changes
            period_sorted = period_data.sort_values('Stride Number').copy()
            period_sorted['Delta'] = period_sorted[stride_col].diff().shift(-1)
            period_sorted = period_sorted[:-1]  # Remove last row
            
            # Separate changes after success vs failure
            success_deltas = period_sorted[period_sorted['Success'] == 1]['Delta'].dropna()
            failure_deltas = period_sorted[period_sorted['Success'] == 0]['Delta'].dropna()
            
            if len(success_deltas) == 0 and len(failure_deltas) == 0:
                ax.text(0.5, 0.5, f'No valid stride changes\\nfor {condition_name.split()[0].lower()} target', 
                       ha='center', va='center', transform=ax.transAxes,
                       fontsize=12, style='italic', color='gray',
                       bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.5))
                ax.set_xlim(-2, 2)
                ax.set_ylim(0, 1)
                return None
            
            # Plot distributions using KDE or fallback methods
            if len(success_deltas) > 0:
                self._plot_distribution(ax, success_deltas, 'green', f'After Success (n={len(success_deltas)})', alpha)
            
            if len(failure_deltas) > 0:
                self._plot_distribution(ax, failure_deltas, 'red', f'After Failure (n={len(failure_deltas)})', alpha)
            
            # Add mean lines and formatting
            if len(success_deltas) > 0:
                ax.axvline(success_deltas.mean(), color='darkgreen', linestyle='--', linewidth=2,
                           label=f'Success Mean: {success_deltas.mean():.3f}')
            
            if len(failure_deltas) > 0:
                ax.axvline(failure_deltas.mean(), color='darkred', linestyle='--', linewidth=2,
                           label=f'Failure Mean: {failure_deltas.mean():.3f}')
            
            ax.axvline(0, color='black', linestyle='-', alpha=0.3, linewidth=1)
            
            # Success rate annotation
            if len(period_sorted) > 0:
                success_rate = period_sorted['Success'].mean()
                ax.text(0.02, 0.98, f'Success Rate: {success_rate:.1%}', 
                       transform=ax.transAxes, fontsize=10, fontweight='bold',
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8),
                       verticalalignment='top', horizontalalignment='left')
            
            ax.legend(loc='upper right', fontsize=9)
            
            # Return statistics
            return {
                f'{const_type}_success_deltas': success_deltas.tolist() if len(success_deltas) > 0 else [],
                f'{const_type}_failure_deltas': failure_deltas.tolist() if len(failure_deltas) > 0 else [],
                f'{const_type}_success_mean': success_deltas.mean() if len(success_deltas) > 0 else None,
                f'{const_type}_failure_mean': failure_deltas.mean() if len(failure_deltas) > 0 else None,
                f'{const_type}_success_rate': period_sorted['Success'].mean(),
                f'{const_type}_n_success': len(success_deltas),
                f'{const_type}_n_failure': len(failure_deltas)
            }
        
        # FIXED: Process all trial types in fixed positions
        all_stats = {'subject_id': subject_id}
        
        for trial_type in trial_types:
            # Get the fixed row and column positions for this trial type
            if trial_type not in layout_mapping:
                continue
                
            row_max, col_max, row_min, col_min = layout_mapping[trial_type]
            
            # Get trial data
            trial_dict = self.data_manager.processed_data[subject_id]['trial_data'].get(trial_type)
            if trial_dict is None:
                df = None
            else:
                df = trial_dict.get('data')
            
            # Check required columns if data exists
            if df is not None and not df.empty:
                required_cols = [stride_col, 'Success', 'Stride Number', 'Target size', 'Constant']
                missing_cols = [col for col in required_cols if col not in df.columns]
                if missing_cols:
                    if show_stats:
                        print(f"Missing columns for {subject_id} {trial_type}: {missing_cols}")
                    df = None
            
            # Get data for both target conditions
            max_const_data = get_period_data(df, 'max') if df is not None else None
            min_const_data = get_period_data(df, 'min') if df is not None else None
            
            # Plot both conditions in their fixed positions
            max_stats = plot_condition_data(
                axes[row_max, col_max], max_const_data, condition_names[0], 'max', trial_type, row_max, col_max
            )
            min_stats = plot_condition_data(
                axes[row_min, col_min], min_const_data, condition_names[1], 'min', trial_type, row_min, col_min
            )
            
            # Combine statistics for this trial type
            trial_stats = {}
            if max_stats:
                trial_stats.update(max_stats)
            if min_stats:
                trial_stats.update(min_stats)
            
            all_stats[trial_type] = trial_stats
        
        # FIXED: Add column headers at the top
        for col, condition_name in enumerate(condition_names):
            fig.text(0.25 + col * 0.5, 0.95, condition_name, ha='center', va='bottom', 
                    fontsize=14, fontweight='bold', transform=fig.transFigure)
        
        # Overall title
        fig.suptitle(f'Stride Length Change Distributions\\n'
                    f'Subject: {subject_id} (Age: {subject_age:.1f} years)', 
                    fontsize=16, fontweight='bold', y=0.98)
        
        plt.tight_layout(rect=[0, 0, 1, 0.94])  # Leave space for title and headers
        
        # Save figure
        if save:
            filename = f"stride_change_{subject_id}_fixed_grid.png"
            save_dir = self.individual_plots_dir / "stride_change_after_success_vs_failure"
            save_dir.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_dir / filename, dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')
        
        plt.close()  # Close to save memory
        
        return all_stats

    def _plot_distribution(self, ax, data, color, label, alpha):
        """Helper method to plot distributions with KDE or fallback."""
        try:
            if len(data) >= 2:
                kde = gaussian_kde(data)
                x_min, x_max = data.min(), data.max()
                x_range = x_max - x_min
                if x_range > 0:
                    x_min -= x_range * 0.1
                    x_max += x_range * 0.1
                else:
                    x_min -= 0.1
                    x_max += 0.1
                
                x_smooth = np.linspace(x_min, x_max, 200)
                y_smooth = kde(x_smooth)
                
                ax.plot(x_smooth, y_smooth, color=color, linewidth=3, alpha=0.8, label=label)
                ax.fill_between(x_smooth, y_smooth, alpha=alpha*0.5, color=color)
            else:
                for i, val in enumerate(data):
                    ax.axvline(val, color=color, alpha=0.7, linewidth=2,
                              label=label if i == 0 else "")
        except Exception:
            # Fallback to histogram
            if len(data) >= 2:
                ax.hist(data, bins=min(10, len(data)), alpha=alpha, color=color, 
                        label=label, density=True, edgecolor='black', linewidth=0.5)
            else:
                for i, val in enumerate(data):
                    ax.axvline(val, color=color, alpha=0.7, linewidth=3,
                              label=label if i == 0 else "")

    def _save_stride_analysis_summary(self, all_stats: Dict) -> None:
        """Save summary of stride analysis to CSV."""
        
        summary_rows = []
        
        for subject_id, subject_data in all_stats.items():
            base_row = {
                'subject_id': subject_id,
                'age': subject_data.get('age', np.nan)
            }
            
            for trial_type in ['vis1', 'invis', 'vis2']:
                if trial_type in subject_data:
                    trial_data = subject_data[trial_type]
                    for condition in ['max', 'min']:
                        row = base_row.copy()
                        row['trial_type'] = trial_type
                        row['condition'] = condition
                        
                        # Add condition-specific data
                        for key, value in trial_data.items():
                            if key.startswith(f'{condition}_'):
                                clean_key = key.replace(f'{condition}_', '')
                                row[clean_key] = value
                        
                        summary_rows.append(row)
        
        if summary_rows:
            summary_df = pd.DataFrame(summary_rows)
            summary_path = self.individual_plots_dir / 'stride_analysis_summary.csv'
            summary_df.to_csv(summary_path, index=False)
            print(f"📊 Stride analysis summary saved to: {summary_path}")

    # ==========================================================================
    # UTILITY METHODS
    # ==========================================================================

    def _add_trendline(self, ax, x, y):
        """Add trendline with correlation coefficient to plot."""
        x_clean = pd.to_numeric(x, errors='coerce')
        y_clean = pd.to_numeric(y, errors='coerce')
        valid = x_clean.notna() & y_clean.notna()
        
        if valid.sum() < 2:
            return
        
        x_vals = x_clean[valid]
        y_vals = y_clean[valid]
        
        try:
            # Fit line
            coeffs = np.polyfit(x_vals, y_vals, 1)
            trendline = np.poly1d(coeffs)
            
            # Calculate correlation
            r, p = pearsonr(x_vals, y_vals)
            
            # Plot trendline
            ax.plot(x_vals, trendline(x_vals), 'r--', alpha=0.8, linewidth=2)
            
            # Add correlation text
            ax.text(0.05, 0.95, f'r² = {r**2:.3f}\\np = {p:.3f}\\nn = {len(x_vals)}', 
                   transform=ax.transAxes,
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                   verticalalignment='top', fontsize=10)
        except Exception as e:
            print(f"Could not add trendline: {e}")

# ==========================================================================
# STREAMLINED PIPELINE CLASS (Updated to use consolidated visualizer)
# ==========================================================================

class StreamlinedMotorLearningPipeline:
    """
    Streamlined pipeline that uses the consolidated StandaloneEnhancedVisualizer.
    All plotting methods have been moved to the visualizer class.
    """
    
    def __init__(self, analysis_instance, output_dir: str = 'motor_learning_analysis'):
        """
        Initialize with your existing analysis instance.
        
        Parameters:
        -----------
        analysis_instance : Your existing analysis object
            Should have .data_manager, .metrics_df attributes
        output_dir : str
            Directory to save all outputs
        """
        
        self.analysis = analysis_instance
        self.data_manager = analysis_instance.data_manager
        self.metrics_df = analysis_instance.metrics_df
        self.output_dir = Path(output_dir)
        
        # Create output directory structure
        self._setup_output_directories()
        
        # Use the consolidated enhanced visualizer
        self.visualizer = StandaloneEnhancedVisualizer(analysis_instance)
        
        # Set configuration
        self.config = self._get_default_config()
        
        # Initialize containers
        self.analysis_results = {}
        self.quality_report = {}
        
        # Analysis timestamp
        self.analysis_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        print(f"🚀 Streamlined Motor Learning Pipeline initialized")
        print(f"📊 Data: {len(self.metrics_df)} subjects with {len(self.metrics_df.columns)} metrics")
        print(f"📁 Output directory: {self.output_dir}")
        print(f"🎨 Using consolidated enhanced visualizer with ALL plotting methods")

    def _setup_output_directories(self):
        """Create organized output directory structure."""
        
        self.output_dir.mkdir(exist_ok=True)
        
        # Create subdirectories
        self.dirs = {
            'figures': self.output_dir / 'figures',
            'individual_plots': self.output_dir / 'figures' / 'individual_plots',
            'population_plots': self.output_dir / 'figures' / 'population_plots',
            'statistical_plots': self.output_dir / 'figures' / 'statistical_plots',
            'reports': self.output_dir / 'reports',
            'exports': self.output_dir / 'exports'
        }
        
        for dir_path in self.dirs.values():
            dir_path.mkdir(parents=True, exist_ok=True)

    def _get_default_config(self) -> Dict:
        """Get default configuration parameters."""
        
        return {
            'motor_noise_threshold': 0.3,
            'success_rate_threshold': 0.68,
            'figure_dpi': 300,
            'alpha_level': 0.05,
            'age_bins': [7, 10, 13, 16, 18],
            'age_labels': ['7-10', '10-13', '13-16', '16-18'],
            'max_individual_subjects': None  # None = all subjects
        }

    def run_complete_analysis(self, include_individual_plots: bool = False) -> Dict:
        """
        Run the complete integrated analysis pipeline using the consolidated visualizer.
        
        Parameters:
        -----------
        include_individual_plots : bool, default False
            Whether to generate individual participant plots (time-consuming)
        
        Returns:
        --------
        Dict : Complete analysis results
        """
        
        print(f"🔄 Starting streamlined analysis pipeline with consolidated visualizer")
        print(f"⏰ Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"🎯 Individual plots: {'Enabled' if include_individual_plots else 'Disabled'}")
        print("=" * 80)
        
        results = {
            'timestamp': self.analysis_timestamp,
            'steps_completed': [],
            'step_results': {},
            'include_individual_plots': include_individual_plots
        }
        
        try:
            # Step 1: Quality control analysis
            print("\\n🔍 STEP 1: QUALITY CONTROL ANALYSIS")
            print("-" * 50)
            step_result = self._step_quality_control()
            results['step_results']['quality_control'] = step_result
            results['steps_completed'].append('quality_control')
            
            # Step 2: Generate ALL visualizations using consolidated visualizer
            print("\\n📈 STEP 2: GENERATING ALL ENHANCED VISUALIZATIONS")
            print("-" * 50)
            viz_result = self.visualizer.generate_all_enhanced_plots(
                include_individual_plots=include_individual_plots
            )
            results['step_results']['visualizations'] = viz_result
            results['steps_completed'].append('visualizations')
            
            # Step 3: Export results
            print("\\n💾 STEP 3: EXPORTING RESULTS")
            print("-" * 50)
            step_result = self._step_export_results()
            results['step_results']['export_results'] = step_result
            results['steps_completed'].append('export_results')
            
        except Exception as e:
            print(f"\\n❌ Pipeline failed: {e}")
            results['error'] = str(e)
            raise
        
        # Final summary
        print("\\n" + "=" * 80)
        print("🎉 STREAMLINED PIPELINE COMPLETED SUCCESSFULLY!")
        print("=" * 80)
        print(f"⏰ Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"✅ Steps completed: {len(results['steps_completed'])}/3")
        print(f"📁 Results saved to: {self.output_dir}")
        
        if not include_individual_plots:
            print("💡 To generate individual plots, rerun with: pipeline.run_complete_analysis(include_individual_plots=True)")
        
        return results

    def _step_quality_control(self) -> Dict:
        """Step 1: Perform quality control analysis."""
        
        try:
            df = self.metrics_df
            
            # Apply motor noise filter
            if 'mot_noise' in df.columns:
                before_filter = len(df)
                filtered_df = df[df['mot_noise'] <= self.config['motor_noise_threshold']]
                after_filter = len(filtered_df)
                
                print(f"🔍 Motor noise filter (≤ {self.config['motor_noise_threshold']}):") 
                print(f"   Before: {before_filter} subjects")
                print(f"   After: {after_filter} subjects") 
                print(f"   Excluded: {before_filter - after_filter} subjects ({100*(before_filter-after_filter)/before_filter:.1f}%)")
                
                self.quality_report['motor_noise_filter'] = {
                    'threshold': self.config['motor_noise_threshold'],
                    'before': before_filter,
                    'after': after_filter,
                    'excluded': before_filter - after_filter,
                    'exclusion_rate': (before_filter - after_filter) / before_filter
                }
            else:
                filtered_df = df
                print("⚠️ No motor noise data available for filtering")
            
            # Basic data summary
            print(f"\\n📊 Final dataset summary:")
            print(f"   Total subjects: {len(filtered_df)}")
            if 'age' in filtered_df.columns:
                print(f"   Age range: {filtered_df['age'].min():.1f} - {filtered_df['age'].max():.1f} years")
                print(f"   Mean age: {filtered_df['age'].mean():.1f} ± {filtered_df['age'].std():.1f} years")
            
            # Count success rate measures
            sr_cols = [col for col in filtered_df.columns if '_sr_' in col]
            print(f"   Success rate measures: {len(sr_cols)}")
            
            self.quality_report.update({
                'final_sample_size': len(filtered_df),
                'n_success_rate_measures': len(sr_cols)
            })
            
            return {
                'success': True,
                'filtered_sample_size': len(filtered_df),
                'quality_report': self.quality_report
            }
            
        except Exception as e:
            print(f"❌ Quality control failed: {e}")
            return {'success': False, 'error': str(e)}

    def _step_export_results(self) -> Dict:
        """Step 3: Export results and generate reports."""
        
        try:
            exported_files = []
            
            # 1. Export metrics CSV
            metrics_file = self.dirs['exports'] / f'metrics_{self.analysis_timestamp}.csv'
            self.metrics_df.to_csv(metrics_file, index=False)
            exported_files.append(str(metrics_file))
            print(f"📊 Metrics CSV: {metrics_file.name}")
            
            # 2. Export quality report
            quality_file = self.dirs['reports'] / f'quality_report_{self.analysis_timestamp}.json'
            with open(quality_file, 'w') as f:
                json.dump(self._make_json_serializable(self.quality_report), f, indent=2)
            exported_files.append(str(quality_file))
            print(f"🔍 Quality report: {quality_file.name}")
            
            # 3. Generate HTML report
            report_file = self._generate_html_report()
            exported_files.append(str(report_file))
            print(f"📄 HTML report: {report_file.name}")
            
            print(f"\\n✅ Exported {len(exported_files)} files")
            
            return {
                'success': True,
                'exported_files': exported_files,
                'n_files': len(exported_files)
            }
            
        except Exception as e:
            print(f"❌ Export failed: {e}")
            return {'success': False, 'error': str(e)}

    def _generate_html_report(self) -> Path:
        """Generate HTML report of the analysis."""
        
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Enhanced Motor Learning Analysis Report</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 40px; }}
                h1 {{ color: #2c3e50; }}
                h2 {{ color: #34495e; border-bottom: 2px solid #ecf0f1; padding-bottom: 10px; }}
                .metric {{ background-color: #f8f9fa; padding: 15px; margin: 10px 0; border-radius: 5px; }}
                .highlight {{ background-color: #e8f5e8; padding: 10px; border-left: 4px solid #28a745; }}
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
            </style>
        </head>
        <body>
            <h1>Enhanced Motor Learning Analysis Report</h1>
            <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            <p><strong>Analysis ID:</strong> {self.analysis_timestamp}</p>
            
            <div class="highlight">
                <h3>🎯 Enhanced Features</h3>
                <ul>
                    <li>✅ ALL plotting methods consolidated in StandaloneEnhancedVisualizer</li>
                    <li>✅ Age vs Success Rates by trial/condition with motor noise coloring</li>
                    <li>✅ Age vs Stride Variability by trial/condition with motor noise coloring</li>
                    <li>✅ Mean Stride Length vs Success Rate by trial/condition with motor noise coloring</li>
                    <li>✅ Age vs Mean Stride Length and Error by trial/condition</li>
                    <li>✅ Motor Noise vs Success Rates by trial/condition with age coloring</li>
                    <li>✅ Comprehensive statistical analysis and feature importance</li>
                    <li>✅ Quality control with motor noise filtering</li>
                    <li>✅ Individual stride change plots (optional)</li>
                </ul>
            </div>
            
            <h2>Dataset Overview</h2>
            <div class="metric">
                <strong>Total Subjects:</strong> {len(self.metrics_df)}<br>
                <strong>Filtered Subjects:</strong> {len(self.metrics_df[self.metrics_df['mot_noise'] <= self.config['motor_noise_threshold']]) if 'mot_noise' in self.metrics_df.columns else 'N/A'}<br>
                <strong>Age Range:</strong> {self.metrics_df['age'].min():.1f} - {self.metrics_df['age'].max():.1f} years<br>
                <strong>Motor Noise Threshold:</strong> {self.config['motor_noise_threshold']}
            </div>
            
            <h2>Quality Control Results</h2>
            <div class="metric">
                {self._format_quality_report_html()}
            </div>
            
            <h2>Generated Visualizations</h2>
            <div class="metric">
                <strong>Enhanced Population Plots:</strong><br>
                • age_vs_success_rates_enhanced.png<br>
                • age_vs_stride_variability_enhanced.png<br>
                • mean_stride_length_vs_success_rate.png<br>
                • age_vs_mean_stride_length_enhanced.png<br>
                • age_vs_mean_error_enhanced.png<br>
                • mean_error_vs_success_rate_enhanced.png<br>
                • motor_noise_vs_success_rates_enhanced.png<br>
                • age_vs_motor_noise.png<br>
                • correlation_matrix.png<br>
                • success_rate_overview.png<br><br>
                
                <strong>Statistical Analysis Plots:</strong><br>
                • feature_importance_analysis.png<br>
                • age_effects_summary.png<br>
                • trial_comparisons.png<br>
                • analysis_dashboard.png<br><br>
                
                <strong>Individual Plots (if enabled):</strong><br>
                • Individual stride change distributions for each participant<br>
                • Stride analysis summary CSV
            </div>
            
            <h2>Files Generated</h2>
            <div class="metric">
                <strong>Output Directory:</strong> {self.output_dir}<br>
                <strong>Figures:</strong> {self.dirs['figures']}<br>
                <strong>Reports:</strong> {self.dirs['reports']}<br>
                <strong>Exports:</strong> {self.dirs['exports']}<br>
                <strong>Consolidated Visualizer:</strong> All plotting methods now in StandaloneEnhancedVisualizer
            </div>
            
            <h2>Usage Instructions</h2>
            <div class="metric">
                <strong>To use the consolidated visualizer directly:</strong><br>
                <code>
                # Create enhanced visualizer<br>
                enhanced_viz = StandaloneEnhancedVisualizer(your_analysis_instance)<br><br>
                
                # Generate all plots at once<br>
                results = enhanced_viz.generate_all_enhanced_plots(include_individual_plots=True)<br><br>
                
                # Or generate specific plot categories<br>
                enhanced_viz.generate_enhanced_population_plots()<br>
                enhanced_viz.generate_statistical_plots()<br>
                enhanced_viz.generate_summary_dashboard()
                </code>
            </div>
        </body>
        </html>
        """
        
        report_file = self.dirs['reports'] / f'analysis_report_{self.analysis_timestamp}.html'
        with open(report_file, 'w') as f:
            f.write(html_content)
        
        return report_file

    def _format_quality_report_html(self) -> str:
        """Format quality report for HTML."""
        
        if not self.quality_report:
            return "Quality report not available"
        
        html_parts = []
        
        if 'motor_noise_filter' in self.quality_report:
            filter_data = self.quality_report['motor_noise_filter']
            html_parts.append(f"""
                <strong>Motor Noise Filtering:</strong><br>
                • Before filtering: {filter_data['before']} subjects<br>
                • After filtering: {filter_data['after']} subjects<br>
                • Exclusion rate: {filter_data['exclusion_rate']*100:.1f}%
            """)
        
        if 'final_sample_size' in self.quality_report:
            html_parts.append(f"<strong>Final Sample Size:</strong> {self.quality_report['final_sample_size']} subjects")
        
        return "<br><br>".join(html_parts) if html_parts else "Quality metrics not available"

    def _make_json_serializable(self, obj):
        """Convert numpy types to Python types for JSON serialization."""
        
        if isinstance(obj, dict):
            return {key: self._make_json_serializable(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self._make_json_serializable(item) for item in obj]
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif pd.isna(obj):
            return None
        else:
            return obj

# ==========================================================================
# USAGE EXAMPLE
# ==========================================================================

def demonstrate_consolidated_visualizer():
    """
    Example of how to use the consolidated enhanced visualizer.
    """
    
    print("🎨 CONSOLIDATED ENHANCED VISUALIZER DEMONSTRATION")
    print("=" * 60)
    print()
    
    print("🔧 Step 1: Create your analysis instance (existing code)")
    print("# analysis = MotorLearningAnalysis(data_manager, metrics_df)")
    print()
    
    print("🎨 Step 2: Create consolidated enhanced visualizer")
    print("# enhanced_viz = StandaloneEnhancedVisualizer(analysis)")
    print()
    
    print("📊 Step 3: Generate ALL plots in one call")
    print("# results = enhanced_viz.generate_all_enhanced_plots(include_individual_plots=True)")
    print()
    
    print("🎯 OR generate specific plot categories:")
    print("# enhanced_viz.generate_enhanced_population_plots()")
    print("# enhanced_viz.generate_statistical_plots()")
    print("# enhanced_viz.generate_summary_dashboard()")
    print()
    
    print("🚀 OR use the streamlined pipeline (uses consolidated visualizer internally)")
    print("# pipeline = StreamlinedMotorLearningPipeline(analysis)")
    print("# results = pipeline.run_complete_analysis(include_individual_plots=True)")
    print()
    
    print("✅ ALL plotting methods are now consolidated in StandaloneEnhancedVisualizer!")
    print("📁 Organized output directories with enhanced plots")
    print("🎨 Motor noise coloring, age effects, trial comparisons, and more")
    print("📊 Statistical analysis plots and comprehensive dashboard")
    print("🎯 Optional individual stride change plots for detailed analysis")

if __name__ == "__main__":
    demonstrate_consolidated_visualizer()

🎨 CONSOLIDATED ENHANCED VISUALIZER DEMONSTRATION

🔧 Step 1: Create your analysis instance (existing code)
# analysis = MotorLearningAnalysis(data_manager, metrics_df)

🎨 Step 2: Create consolidated enhanced visualizer
# enhanced_viz = StandaloneEnhancedVisualizer(analysis)

📊 Step 3: Generate ALL plots in one call
# results = enhanced_viz.generate_all_enhanced_plots(include_individual_plots=True)

🎯 OR generate specific plot categories:
# enhanced_viz.generate_enhanced_population_plots()
# enhanced_viz.generate_statistical_plots()
# enhanced_viz.generate_summary_dashboard()

🚀 OR use the streamlined pipeline (uses consolidated visualizer internally)
# pipeline = StreamlinedMotorLearningPipeline(analysis)
# results = pipeline.run_complete_analysis(include_individual_plots=True)

✅ ALL plotting methods are now consolidated in StandaloneEnhancedVisualizer!
📁 Organized output directories with enhanced plots
🎨 Motor noise coloring, age effects, trial comparisons, and more
📊 Statistical anal

In [10]:
# CORRECTED USAGE CODE
# The issue was passing config instead of a string path

METADATA_PATH = "muh_metadata.csv"
DATA_ROOT_DIR = "muh_data/"

# 1. Create centralized config
config = Config('analysis')

# 2. Create data manager with config
data_manager = MotorLearningDataManager(
    METADATA_PATH, 
    DATA_ROOT_DIR, 
    config=config, 
    debug=True
)

data_manager = data_manager.filter_trials(required_trial_types=['vis1', 'invis', 'vis2'])

# 3. Calculate metrics
metrics_calculator = MetricsCalculator(data_manager)
metrics_df = metrics_calculator.calculate_all_metrics()

# 4. Create analysis with shared config
analysis = MotorLearningAnalysis(data_manager, metrics_df)

print(f"✅ Analysis ready! {len(metrics_df)} subjects loaded.")
print(f"📁 All outputs will be saved to: {config.BASE_OUTPUT_DIR}")
print(f"🎯 Visualizer initialized successfully: {hasattr(analysis.visualizer, 'individual_plots_dir')}")

📁 Config initialized with base directory: analysis
   📊 Figures: analysis\figures
   📋 Reports: analysis\reports
   💾 Exports: analysis\exports
🧮 Calculating metrics for 66 subjects...
   Processed 20/66 subjects...
   Processed 40/66 subjects...
   Processed 60/66 subjects...
✅ Successfully calculated metrics for 66 subjects
📊 Age range: 7.2 - 17.9 years
🎯 Success rate columns created: ['vis1_sr_max_const', 'vis1_sr_min_const', 'invis_sr_max_const', 'invis_sr_min_const', 'vis2_sr_max_const', 'vis2_sr_min_const']
📊 Filtered to 64/66 subjects (motor noise ≤ 0.3)
📊 Using 64/66 subjects (motor noise ≤ 0.3)
✅ Enhanced visualizer initialized with consolidated plotting methods
📁 Individual plots: analysis\figures\individual_plots
📁 Population plots: analysis\figures\population_plots
📁 Statistical plots: analysis\figures\statistical_plots
✅ Analysis ready! 66 subjects loaded.
📁 All outputs will be saved to: analysis
🎯 Visualizer initialized successfully: True


In [11]:
# 5. CORRECTED: Create integrated pipeline with STRING output directory
# Option 1: Use the same directory as config
pipeline = StreamlinedMotorLearningPipeline(analysis, str(config.BASE_OUTPUT_DIR))

# OR Option 2: Use a different directory name
# pipeline = IntegratedMotorLearningPipeline(analysis, 'integrated_analysis_output')

print(f"✅ Pipeline ready!")

# 6. Run the full pipeline
print("\n🚀 Running integrated analysis pipeline...")
pipeline_results = pipeline.run_complete_analysis(include_individual_plots=False)

📊 Using 64/66 subjects (motor noise ≤ 0.3)
✅ Enhanced visualizer initialized with consolidated plotting methods
📁 Individual plots: analysis\figures\individual_plots
📁 Population plots: analysis\figures\population_plots
📁 Statistical plots: analysis\figures\statistical_plots
🚀 Streamlined Motor Learning Pipeline initialized
📊 Data: 66 subjects with 58 metrics
📁 Output directory: analysis
🎨 Using consolidated enhanced visualizer with ALL plotting methods
✅ Pipeline ready!

🚀 Running integrated analysis pipeline...
🔄 Starting streamlined analysis pipeline with consolidated visualizer
⏰ Started at: 2025-06-03 11:48:39
🎯 Individual plots: Disabled
\n🔍 STEP 1: QUALITY CONTROL ANALYSIS
--------------------------------------------------
🔍 Motor noise filter (≤ 0.3):
   Before: 66 subjects
   After: 64 subjects
   Excluded: 2 subjects (3.0%)
\n📊 Final dataset summary:
   Total subjects: 64
   Age range: 7.2 - 17.9 years
   Mean age: 12.4 ± 3.2 years
   Success rate measures: 6
\n📈 STEP 2: GENE

C:\Users\castle\AppData\Local\Temp\ipykernel_9452\3420013994.py:1202: UserWarning: Glyph 9989 (\N{WHITE HEAVY CHECK MARK}) missing from font(s) DejaVu Sans.
  plt.savefig(dashboard_file, dpi=getattr(self.config, 'FIGURE_DPI', 300), bbox_inches='tight')


   Generated summary dashboard: analysis_dashboard_20250603_114847.png

🎉 ENHANCED PLOTTING COMPLETED SUCCESSFULLY!
⏰ Completed at: 2025-06-03 11:48:48
📊 Total plots generated: 13
   • Individual plots: 0
   • Population plots: 10
   • Statistical plots: 3
📁 Files saved to: analysis
💡 To generate individual plots, use: generate_all_enhanced_plots(include_individual_plots=True)
\n💾 STEP 3: EXPORTING RESULTS
--------------------------------------------------
📊 Metrics CSV: metrics_20250603_114839.csv
🔍 Quality report: quality_report_20250603_114839.json
❌ Export failed: 'charmap' codec can't encode character '\U0001f3af' in position 1093: character maps to <undefined>
\n================================================================================
🎉 STREAMLINED PIPELINE COMPLETED SUCCESSFULLY!
⏰ Completed at: 2025-06-03 11:48:48
✅ Steps completed: 3/3
📁 Results saved to: analysis
💡 To generate individual plots, rerun with: pipeline.run_complete_analysis(include_individual_plots=True)


In [ ]:

prediction_results = pipeline.predict_stride_error_simple(pipeline.metrics_df)
pipeline.plot_prediction_results_simple(prediction_results)


In [ ]:
#%run powerpoint_auto_generator.py